**The next block defines the functions used:**

In [ ]:
from openai import OpenAI
import os
import tiktoken
import re

client = OpenAI(api_key='')

def openai_api_function(QQ, T=0, seed=999, model="gpt-4o", syst='You are an expert in the alloy field of Materials Science and Engineering', timeout=240):
    tkn = 4000
    Q = []
    roles = ['user', 'assistant']
    Q.append({'role': 'system', 'content': syst})
    
    for qqq in enumerate(QQ):
        Q.append({'role': roles[qqq[0] % 2], 'content': qqq[1]})

    while True:
        try:
            response = client.chat.completions.create(
                model=model,
                messages=Q,
                seed=seed,
                temperature=T,
                max_tokens=tkn,
                frequency_penalty=0,
                presence_penalty=0,
                timeout=timeout
            )
            break
        except Exception as e:
            print("An error occurred:", e)
            if 'Please reduce the length of the messages' in str(e):
                print('TRUNCATING')
                if 'Identify the top three most significant alloy properties discussed in the text' in Q[1]["content"]:
                    print(Q.pop(3))
                    print(Q.pop(3))
                else:
                    print("The paper is no longer in the conversation!")
                    print(Q.pop(1))
                    print(Q.pop(1))
            elif 'per min' in str(e):
                print("Sleeping for 15 sec.")
                from time import sleep
                sleep(15)

    return (QQ, response.choices[0].message.content)

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def process_list(input_list):
    # Remove spaces at the beginning and end of each string
    input_list = [s.strip() for s in input_list]
    
    # Convert each string to lowercase
    input_list = [s.lower() for s in input_list]
    
    # Remove the '.' at the end of each string, if present
    input_list = [s[:-1] if s.endswith('.') else s for s in input_list]
    
    return input_list

# This function is used to remove microstructure terms in Properties list:
def check_unimportant_property(i):
    possible_term = ['precipitate', 'precipitation', 'lattice', 'phase', 'austenite', 'ferrite', 'martensite', 'lath', 'dislocation', 'grain', 'twin', 'stacking fault', 'microstructure', 'morphology', 'solution', 'single-phase', 'structure']
    for j in possible_term:
        if j in i:
            return True
    return False

def check_unimportant_microstructure(i):
    possible_term = ['peak', 'Peak', 'column', 'superalloy', 'Superalloy', 'solution strengthening']
    for j in possible_term:
        if j in i:
            return True
    return False

# This function is used to define whether it is an unimportant processing method in Processing list: 
def check_unimportant_processing(i):
    if ('remelt' in i) or ('Remelt' in i):
        return False
    possible_term = ['test', 'analysis', 'melt', 'Melt', 'cast', 'Cast', 'APT', 'TEM', 'FIB', 'XRD', 'ND', 'polish', 'Polish'] 
    for j in possible_term:
        if j in i:
            return True
    return False

def separating_properties(input_data):
    # Split the input into lines and process each line
    lines = input_data.strip().split('\n')
    headers = lines[0]
    data_rows = lines[1:]

    # Initialize the new data structure
    new_data_rows = []

    for row in data_rows:
        # Handle potential extra spaces and split by '|'
        cells = [cell.strip() for cell in row.strip().split('|')]
        cells = [cell for cell in cells if cell]  # Remove empty cells

        # Split properties in the last cell
        properties = cells[-1].split(';')
        for prop in properties:
            new_row = cells[:-1] + [prop.strip()]
            new_data_rows.append('| ' + ' | '.join(new_row) + ' |')

    # Prepare the output
    output = headers + '\n' + '\n'.join(new_data_rows)
    return output


**The next block defines the paper set:**

In [12]:
paper_list = []
paper_path = 'Text_Corrosion'
# Set a counter for the number of papers used:
counter = 0
for i in os.listdir(paper_path):
    if i.endswith('.txt'):
        total_path = paper_path + '/' + i
        with open(total_path, 'r', encoding='utf-8') as f:
            paper_list.append(f.read())
len(paper_list)

25

**The following blocks are the prompts:**

In [13]:
prompt_for_paper_classification = '''In the field of corrosion and passivation studies on high-entropy alloys (HEAs), scholarly papers generally fall into one of three research paradigms:

**Experimental/Practical Research:**  
These papers focus on empirical studies of corrosion or passivation behavior. They emphasize experimental techniques such as electrochemical testing (e.g., potentiodynamic polarization, EIS), surface characterization (e.g., SEM, XPS), and alloy processing methods like arc melting, heat treatment, or coating deposition. While some computational insights might be included, the central contribution is based on experimental data that demonstrate how processing or composition affects corrosion resistance or passive film formation.

**Theoretical/Computational Research:**  
These papers rely exclusively on computational or theoretical methods to investigate corrosion or passivation phenomena. This includes density functional theory (DFT), Monte Carlo simulations, molecular dynamics, or thermodynamic modeling to predict surface energies, adsorption behavior, charge transfer properties, or passive film stability. No actual alloy processing or wet-lab testing is involved. The focus is on predictive models and simulations that offer theoretical insight into corrosion-related behavior in HEAs.

**Mixed Research:**  
These papers combine both experimental and computational approaches. They might use modeling (e.g., DFT, machine learning, thermodynamic calculations) to guide alloy design or explain electrochemical behavior, followed by experimental validation through corrosion testing and surface characterization. Both the computational and experimental components are substantial and support one another to provide a comprehensive understanding of corrosion/passivation mechanisms.

**Please evaluate the attached paper and determine which category it best fits. Provide a clear classification and a brief justification, referencing aspects such as the research methods used (experimental vs. computational), the main focus of the study, the type of data presented (e.g., polarization curves, surface analysis, simulation outputs), and how the conclusions contribute to understanding or improving corrosion resistance in HEAs.**

**Format for Response:**  
Category: [Experimental/Practical, Theoretical/Computational, or Mixed]  
Reasons: [Concise explanation referencing research methods, primary focus, type of data (experimental or simulation), and the nature of the conclusions.]
'''

In [14]:
prompts_for_experimental_type_papers = ['''
Identify the alloy's significant corrosion-resistance properties discussed in the text, focusing on oxide species’ presence, enhanced corrosion resistance, multiphase presence, testing environment (solution chemistry and pH) and alloy composition. Include only those properties explicitly mentioned in the text, avoiding any not directly addressed. Prioritize information provided in the Abstract and Conclusion sections. Ensure each property description is both concise and precise. Avoid using any microstructure-related terms in your answer. Use semicolons to separate your answers. Please make sure your answer here is no more than 30 words.''' , # Get the target property.                           
'''
Comprehensively summarize structural characteristics relevant to the alloy's corrosion properties or passivation mechanisms as discussed in the paper. You are encouraged to particularly consider: multiphase microstructures, elemental segregation, localized corrosion, interfaces; distribution of atoms, etc., but not exclusively. Keep every structural characteristics as concise as possible, and use semicolons to separate your responses. Please make sure your answer here is no more than 30 words.''', # Get the Microstructure of the material.
'''
First, provide the composition of this alloy. Then, perform a detailed analysis to determine if there are any additions of specific chemical elements that significantly affect the alloy's performance or structure. It is important to note any prefixes or suffixes associated with these elements, as they can indicate different forms or concentrations (e.g., 'trace', 'high', 'ultrafine', etc.). These details are crucial for preserving the accuracy of the information presented in the paper. Ensure the composition and key elements are distinguished from the structure discussed previously; in other words, they should not overlap with answers from earlier structure questions. Next, identify and list the phase composition, where each phase's chemical makeup is specified and separated by semicolons. Be specific and avoid general descriptors like 'mixed phases' — name and detail each if available. Provide only the results, without any additional information or explanation. Respond in this format: Composition: YOUR ANSWER; Key elements addition: YOUR ANSWER; Phase Composition YOUR ANSWER. If no such key element is identified, state 'N/A'.''', # Get the composition and key elements(if any) of the alloy.
'''
1. Construct a table linking structure, associated passivation and corrosion mechanisms, and resultant corrosion-resistant properties. Consider only the structures and properties outlined in the prerequisites, without altering them when completing the table. The mechanisms described in the paper should always take precedence. Present your findings in a table format with three columns: Structure, Passivation & Corrosion Mechanism (structure & property) , and Property. Include every listed structure and describe the mechanism in detail to prevent ambiguity.
2. To ensure clarity and accuracy in indicating the source of each mechanism (except when N/A), adhere to the following guidelines in your responses: For Passivation & Corrosion Mechanisms Directly from the Original Text: If the explanation or mechanism you provide is directly taken or closely paraphrased from the original text, indicate this using the format: "XXX (From text)". For Passivation & Corrosion Mechanisms Derived from Your Own Knowledge Base: If the explanation or mechanism is not found in the original text and instead comes from your understanding or external knowledge, use the format: "XXX (From knowledge base)". Please strictly follow this format for each mechanism you explain. ''', # Get the mechanism between the target property and the Microstructure.

'''List the key processing or manufacturing techniques explicitly stated in the text provided for this alloy; do not base your answers on typical experience. Avoid mentioning details about micro/macro structures. Use semicolons to separate your answers. Additional Requirements: 1. For the melting process, always combine the use of protective gas with the melting action into a single description, such as 'melting in a protective gas environment'; do not separate these elements. 2. Exclude any processing methods that are hypothetical or proposed as future improvements. Only include those explicitly employed in the current context of the alloy's manufacturing or processing. 3. Focus solely on manufacturing methods, excluding analytical methods like linear polarization resistance, AC impedance measurements, and microscopic analyses.
''', # Get the processing method of the material.

'''From the list of processing methods of this alloy discussed previously, identify the method(s) most central to the paper's discussion. For instance, if process A is highlighted frequently, consider it the primary method. Exclude any pre-processing techniques such as arc-melting. Additionally, provide a brief explanation as to why this method, for example, process A, is emphasized in the paper over others, based on the frequency of its mention and its relevance to the key findings or conclusions. This will help to understand the context and significance of the chosen method(s) in the overall study. Ensure to exclude any processing methods not explicitly mentioned in the text.
RETURN ME STRICTLY WITH THIS FORMAT: The final processing methods are: YOUR ANSWER; The explanations are: YOUR EXPLANATIONS.''', # Refine the processing method of the material.

'''
1. Construct a table that maps structures to their influential processing methods and the mechanisms responsible for their formation. Frame the relationship as follows: "The [manufacturing method] yields [structure] via the [mechanism]." For instance, "Chemical passivation produces corrosion-resistant structures through the formation of a stable oxide layer."  Ensure a logical and coherent connection among these three elements. The table should consist of three columns titled 'Processing', 'Passivation & Corrosion Mechanism (Processing & structure)', and 'structure'. Incorporate all relevant structures, omitting those not pertinent to the discussion. Consider only the processing methods specified in the 'processing list'. Each structure must be matched with pertinent processing methods and their mechanisms, emphasizing a clear and logical link among them. The table's content should be concise, focusing exclusively on the interplay between processing methods, mechanisms, and structures.
2. For clarity and precision in citing the origin of each mechanism (except when not applicable), adhere to the guidelines below in your responses: For 'Passivation & Corrosion  mechanisms directly quoted or closely paraphrased from the original text, use the format: "XXX (From text)". For 'Passivation & Corrosion  mechanisms derived from your own knowledge or external sources, use the format: "XXX (From knowledge base)". Strictly follow this format for every mechanism explanation provided.''', # Get the mechanism between the structure and the processing.

"Review the provided list that details the composition/key elements/phase composition of the alloy you previously identified. Analyze and identify any components in the list that significantly influence the alloy's passivation and corrosion performance. If any such components are identified, add them to the 'Processing' column of the final table's corresponding row. Please do not overwrite the content in the 'Processing' column of the original table unless it is marked as N/A, and refrain from creating a new column for composition/key elements.", # Try to add composition and key elements into the Processing column.
                                        
'''Merge the two extracted tables into a single comprehensive table, with the 'structure' column serving as the primary criterion for merging. Here are the specific instructions:
1. Merging Based on structure: Combine the tables into one, aligning entries based on similarities or matches in the 'structure' column.
2. Handling Non-Matching structures: For entries in the 'structure' column that do not match, analyze their relevance and importance based on the context provided by the source papers.
3. Decision on Exclusion: After analysis, decide whether the non-matching structure entries are essential to the merged table's overall content and purpose. If not, consider removing these entries to maintain coherence and relevance.
4. Columns of the final table: Include Processing, Passivation & Corrosion Mechanism (Processing & structure), structure, Passivation & Corrosion Mechanism (structure & Property), and Property.
5. Ensure the final table is complete, with no gaps. Adjust content as needed for accuracy and consistency with the original texts.
6. Mark all uncertain positions as 'N/A'. Replace all blank positions with 'N/A'.
7. Prune the table: Examine each row; remove rows with three or more 'N/A' entries. Keep rows with fewer than three 'N/A' entries. If significant information is lost due to pruning, incorporate essential details into other rows, ensuring data coherence and rationale are preserved.
8. After completing the above steps, review the document to assess the content's rationality and identify any extraneous content for removal.
9. Only the final, corrected table is required; details about the process are not needed. 
The goal is to produce a unified table that accurately reflects the combined data, focusing on maintaining relevance and coherence, especially regarding the 'structure' column.''', # Merge the two tables.

'''Add one more column to the right of every row, specifying the number of this paper. Example of the result: 
| Processing         | Passivation & Corrosion Mechanism (Processing & structure)| structure | Passivation & Corrosion Mechanism (structure & Property)    | Property  | Paper Number |
|--------------------|--------------------------|------------------|-----------------------|---------------|----------|
| XXXXXXXXXXX     | XXXXXXXXXXX                | XXXXXXXXXXX          | XXXXXXXXXXX        |XXXXXXXXXXX | [4]    |
| XXXXXXXXXXX    | XXXXXXXXXXX                 | XXXXXXXXXXX       | XXXXXXXXXXX           | XXXXXXXXXXX | [4]    |'''
 # Add the paper number to the table.
]


In [15]:
prompts_for_computational_type_papers = [
''' Identify the alloy's significant corrosion-resistance properties discussed in the text, focusing on oxide species’ presence, enhanced corrosion resistance, multiphase presence, testing environment (solution chemistry and pH) and alloy composition. Include only those properties explicitly mentioned in the text, avoiding any not directly addressed. Prioritize information provided in the Abstract and Conclusion sections. Ensure each property description is both concise and precise. Avoid using any microstructure-related terms in your answer. Use semicolons to separate your answers. Please make sure your answer here is no more than 30 words.''' ,  # Get the target property.                           
                          
'''Comprehensively summarize structural characteristics relevant to the alloy's corrosion properties or passivation mechanisms as discussed in the paper. You are encouraged to particularly consider: multiphase microstructures, elemental segregation, localized corrosion, interfaces; distribution of atoms, etc., but not exclusively. Keep every structural characteristics as concise as possible, and use semicolons to separate your responses. Please make sure your answer here is no more than 30 words.''', # Get the Microstructure of the material.


'''
First, provide the composition of this alloy. Then, perform a detailed analysis to determine if there are any additions of specific chemical elements that significantly affect the alloy's performance or structure. It is important to note any prefixes or suffixes associated with these elements, as they can indicate different forms or concentrations (e.g., 'trace', 'high', 'ultrafine', etc.). These details are crucial for preserving the accuracy of the information presented in the paper. Ensure the composition and key elements are distinguished from the structure discussed previously; in other words, they should not overlap with answers from earlier structure questions. Next, identify and list the phase composition, where each phase's chemical makeup is specified and separated by semicolons. Be specific and avoid general descriptors like 'mixed phases' — name and detail each if available. Provide only the results, without any additional information or explanation. Respond in this format: Composition: YOUR ANSWER; Key elements addition: YOUR ANSWER; Phase Composition YOUR ANSWER. If no such key element is identified, state 'N/A'.''', # Get the composition and key elements(if any) of the alloy.

''' 1. Construct a table linking structure, associated passivation and corrosion mechanisms, and resultant corrosion-resistant properties. Consider only the structures and properties outlined in the prerequisites, without altering them when completing the table. The mechanisms described in the paper should always take precedence. Present your findings in a table format with three columns: Structure, Passivation & Corrosion Mechanism (structure & property) , and Property. Include every listed structure and describe the mechanism in detail to prevent ambiguity.
2. To ensure clarity and accuracy in indicating the source of each mechanism (except when N/A), adhere to the following guidelines in your responses: For Passivation & Corrosion Mechanisms Directly from the Original Text: If the explanation or mechanism you provide is directly taken or closely paraphrased from the original text, indicate this using the format: "XXX (From text)". For Passivation & Corrosion Mechanisms Derived from Your Own Knowledge Base: If the explanation or mechanism is not found in the original text and instead comes from your understanding or external knowledge, use the format: "XXX (From knowledge base)". Please strictly follow this format for each mechanism you explain. ''', # Get the mechanism between the target property and the Microstructure. 

''' 1. Construct a table that maps structures to their influential processing methods and the mechanisms responsible for their formation. Frame the relationship as follows: "The [manufacturing method] yields [structure] via the [mechanism]." For instance, "Chemical passivation produces corrosion-resistant structures through the formation of a stable oxide layer."  Ensure a logical and coherent connection among these three elements. The table should consist of three columns titled 'Processing', 'Passivation & Corrosion Mechanism (Processing & structure)', and 'structure'. Incorporate all relevant structures, omitting those not pertinent to the discussion. Consider only the processing methods specified in the 'processing list'. Each structure must be matched with pertinent processing methods and their mechanisms, emphasizing a clear and logical link among them. The table's content should be concise, focusing exclusively on the interplay between processing methods, mechanisms, and structures.
2. For clarity and precision in citing the origin of each mechanism (except when not applicable), adhere to the guidelines below in your responses: For 'Passivation & Corrosion  mechanisms directly quoted or closely paraphrased from the original text, use the format: "XXX (From text)". For 'Passivation & Corrosion  mechanisms derived from your own knowledge or external sources, use the format: "XXX (From knowledge base)". Strictly follow this format for every mechanism explanation provided.''', # Get the mechanism between the structure and the processing. # Get the mechanism between the Microstructure and the composition/key elements.

'''Merge the two extracted tables into a single comprehensive table, with the 'structure' column serving as the primary criterion for merging. Here are the specific instructions:

1. Merging Based on structure: Combine the tables into one, aligning entries based on similarities or matches in the 'structure' column.
2. Handling Non-Matching structures: For entries in the 'structure' column that do not match, analyze their relevance and importance based on the context provided by the source papers.
3. Decision on Exclusion: After analysis, decide whether the non-matching structure entries are essential to the merged table's overall content and purpose. If not, consider removing these entries to maintain coherence and relevance.
4. Columns of the final table: Include Processing, Passivation & Corrosion Mechanism (Processing & structure), structure, Passivation & Corrosion Mechanism (structure & Property), and Property.
5. Ensure the final table is complete, with no gaps. Adjust content as needed for accuracy and consistency with the original texts.
6. Mark all uncertain positions as 'N/A'. Replace all blank positions with 'N/A'.
7. Prune the table: Examine each row; remove rows with three or more 'N/A' entries. Keep rows with fewer than three 'N/A' entries. If significant information is lost due to pruning, incorporate essential details into other rows, ensuring data coherence and rationale are preserved.
8. After completing the above steps, review the document to assess the content's rationality and identify any extraneous content for removal.
9. Only the final, corrected table is required; details about the process are not needed. 
The goal is to produce a unified table that accurately reflects the combined data, focusing on maintaining relevance and coherence, especially regarding the 'structure' column.''', # Merge the two tables.

'''1. Add an additional column to the right of every row, specifying the number of this paper.
2. If 'Processing' still shows in the table, replace it to 'Composition/key elements'. The content within the column should remain unchanged.
3. If '| Passivation & Corrosion  Mechanism (Processing & structure)' still shows in the table, replace it to '| Passivation & Corrosion  Mechanism (Composition/key elements & structure)'. The content within the column should remain unchanged.
Example of the result:
| Composition/key elements | | Passivation & Corrosion Mechanism (Composition/key elements & structure)| structure  | Passivation & Corrosion Mechanism (structure & Property)    | Property  | Paper Number |
|--------------------|--------------------------|------------------|-----------------------|---------------|----------|
| XXXXXXXXXXX     | XXXXXXXXXXX                | XXXXXXXXXXX          | XXXXXXXXXXX        |XXXXXXXXXXX | [4]    |
| XXXXXXXXXXX    | XXXXXXXXXXX                 | XXXXXXXXXXX       | XXXXXXXXXXX           | XXXXXXXXXXX | [4]    |'''

 # Add the paper number to the table.
]

In [17]:
prompts_for_mixed_type_papers = ['''As discussed earlier, this paper is comprehensive, combining both experimental techniques and computational models. Please divide the paper into three sections: A) Computational model and theoretical analysis of the material, B) Material synthesis and manufacturing, and C) Material's performance testing. Keep in mind not to submit the results of your division.
Identify the alloy's significant corrosion-resistance properties discussed in the text, focusing on oxide species’ presence, enhanced corrosion resistance, multiphase presence, testing environment (solution chemistry and pH) and alloy composition. Include only those properties explicitly mentioned in the text, avoiding any not directly addressed. Prioritize information provided in the Abstract and Conclusion sections. Ensure each property description is both concise and precise. Avoid using any microstructure-related terms in your answer. Use semicolons to separate your answers. Please make sure your answer here is no more than 30 words.''' , # Get the target property.                           
'''Comprehensively summarize structural characteristics relevant to the alloy's corrosion properties or passivation mechanisms as discussed in the paper. You are encouraged to particularly consider: multiphase microstructures, elemental segregation, localized corrosion, interfaces; distribution of atoms, etc., but not exclusively. Keep every structural characteristics as concise as possible, and use semicolons to separate your responses. Please make sure your answer here is no more than 30 words.''', # Get the Microstructure of the material.
'''First, provide the composition of this alloy. Then, perform a detailed analysis to determine if there are any additions of specific chemical elements that significantly affect the alloy's performance or structure. It is important to note any prefixes or suffixes associated with these elements, as they can indicate different forms or concentrations (e.g., 'trace', 'high', 'ultrafine', etc.). These details are crucial for preserving the accuracy of the information presented in the paper. Ensure the composition and key elements are distinguished from the structure discussed previously; in other words, they should not overlap with answers from earlier structure questions. Next, identify and list the phase composition, where each phase's chemical makeup is specified and separated by semicolons. Be specific and avoid general descriptors like 'mixed phases' — name and detail each if available. Provide only the results, without any additional information or explanation. Respond in this format: Composition: YOUR ANSWER; Key elements addition: YOUR ANSWER; Phase Composition YOUR ANSWER. If no such key element is identified, state 'N/A'.''', # Get the composition and key elements(if any) of the alloy.

                                 
'''
1. Construct a table linking structure, associated passivation and corrosion mechanisms, and resultant corrosion-resistant properties. Consider only the structures and properties outlined in the prerequisites, without altering them when completing the table. The mechanisms described in the paper should always take precedence. Present your findings in a table format with three columns: Structure, Passivation & Corrosion Mechanism (structure & property) , and Property. Include every listed structure and describe the mechanism in detail to prevent ambiguity.
2. To ensure clarity and accuracy in indicating the source of each mechanism (except when N/A), adhere to the following guidelines in your responses: For Passivation & Corrosion Mechanisms Directly from the Original Text: If the explanation or mechanism you provide is directly taken or closely paraphrased from the original text, indicate this using the format: "XXX (From text)". For Passivation & Corrosion Mechanisms Derived from Your Own Knowledge Base: If the explanation or mechanism is not found in the original text and instead comes from your understanding or external knowledge, use the format: "XXX (From knowledge base)". Please strictly follow this format for each mechanism you explain ''', # Get the mechanism between the target property and the Microstructure.

'''Only focus on section B, list the key processing or manufacturing techniques explicitly stated in the text provided for this alloy; do not base your answers on typical experience. . Avoid mentioning details about micro/macro structures. Use semicolons to separate your answers. Additional Requirements: 1. For the melting process, always combine the use of protective gas with the melting action into a single description, such as 'melting in a protective gas environment'; do not separate these elements. 2. Exclude any processing methods that are hypothetical or proposed as future improvements. Only include those explicitly employed in the current context of the alloy's manufacturing or processing. 3. Focus solely on manufacturing methods, excluding analytical methods like linear polarization resistance, AC impedance measurements, and microscopic analyses.
''', # Get the processing method of the material.


''' From the list of processing methods of this alloy discussed previously, identify the method(s) most central to the paper's discussion. For instance, if process A is highlighted frequently, consider it the primary method. Exclude any pre-processing techniques such as arc-melting. Additionally, provide a brief explanation as to why this method, for example, process A, is emphasized in the paper over others, based on the frequency of its mention and its relevance to the key findings or conclusions. This will help to understand the context and significance of the chosen method(s) in the overall study. Ensure to exclude any processing methods not explicitly mentioned in the text.
RETURN ME STRICTLY WITH THIS FORMAT: The final processing methods are: YOUR ANSWER; The explanations are: YOUR EXPLANATIONS.''', # Refine the processing method of the material.

'''
1. Construct a table that maps structures to their influential processing methods and the mechanisms responsible for their formation. Frame the relationship as follows: "The [manufacturing method] yields [structure] via the [mechanism]." For instance, "Chemical passivation produces corrosion-resistant structures through the formation of a stable oxide layer."  Ensure a logical and coherent connection among these three elements. The table should consist of three columns titled 'Processing', 'Passivation & Corrosion Mechanism (Processing & structure)', and 'structure'. Incorporate all relevant structures, omitting those not pertinent to the discussion. Consider only the processing methods specified in the 'processing list'. Each structure must be matched with pertinent processing methods and their mechanisms, emphasizing a clear and logical link among them. The table's content should be concise, focusing exclusively on the interplay between processing methods, mechanisms, and structures.
2. For clarity and precision in citing the origin of each mechanism (except when not applicable), adhere to the guidelines below in your responses: For 'Passivation & Corrosion  mechanisms directly quoted or closely paraphrased from the original text, use the format: "XXX (From text)". For 'Passivation & Corrosion  mechanisms derived from your own knowledge or external sources, use the format: "XXX (From knowledge base)". Strictly follow this format for every mechanism explanation provided.''', # Get the mechanism between the structure and the processing.

" Review the provided list that details the composition/key elements/phase composition of the alloy you previously identified. Analyze and identify any components in the list that significantly influence the alloy's passivation and corrosion performance. If any such components are identified, add them to the 'Processing' column of the final table's corresponding row. Please do not overwrite the content in the 'Processing' column of the original table unless it is marked as N/A, and refrain from creating a new column for composition/key elements.", # Try to add composition and key elements into the Processing column.                                 
'''Merge the two extracted tables into a single comprehensive table, with the 'structure' column serving as the primary criterion for merging. Here are the specific instructions:
1. Merging Based on structure: Combine the tables into one, aligning entries based on similarities or matches in the 'structure' column.
2. Handling Non-Matching structures: For entries in the 'structure' column that do not match, analyze their relevance and importance based on the context provided by the source papers.
3. Decision on Exclusion: After analysis, decide whether the non-matching structure entries are essential to the merged table's overall content and purpose. If not, consider removing these entries to maintain coherence and relevance.
4. Columns of the final table: Include Processing, Passivation & Corrosion Mechanism (Processing & structure), structure, Passivation & Corrosion Mechanism (structure & Property), and Property.
5. Ensure the final table is complete, with no gaps. Adjust content as needed for accuracy and consistency with the original texts.
6. Mark all uncertain positions as 'N/A'. Replace all blank positions with 'N/A'.
7. Prune the table: Examine each row; remove rows with three or more 'N/A' entries. Keep rows with fewer than three 'N/A' entries. If significant information is lost due to pruning, incorporate essential details into other rows, ensuring data coherence and rationale are preserved.
8. After completing the above steps, review the document to assess the content's rationality and identify any extraneous content for removal.
9. Only the final, corrected table is required; details about the process are not needed. 
The goal is to produce a unified table that accurately reflects the combined data, focusing on maintaining relevance and coherence, especially regarding the 'structure' column.''', # Merge the two tables.

'''Add one more column to the right of every row, specifying the number of this paper. Example of the result: 
| Processing         | Passivation & Corrosion Mechanism (Processing & structure)| structure | Passivation & Corrosion Mechanism (structure & Property)    | Property  | Paper Number |
|--------------------|--------------------------|------------------|-----------------------|---------------|----------|
| XXXXXXXXXXX     | XXXXXXXXXXX                | XXXXXXXXXXX          | XXXXXXXXXXX        |XXXXXXXXXXX | [4]    |
| XXXXXXXXXXX    | XXXXXXXXXXX                 | XXXXXXXXXXX       | XXXXXXXXXXX           | XXXXXXXXXXX | [4]    |'''
# Add the paper number to the table.

                  ]

**Main loop for system chart extraction:**

In [22]:
import os

paper_path = 'Text_Corrosion'
file_name_system_chart_alloy = 'System_Table_HEA_Corrosion'
table_saver = {}
citation_saver = {}

# Main loop for processing individual paper files
for count_for_papers, filename in enumerate(sorted(os.listdir(paper_path)), start=1):
    if not filename.endswith('.txt'):
        continue

    full_path = os.path.join(paper_path, filename)
    with open(full_path, 'r', encoding='utf-8') as f:
        paper = f.read()
    # Defining some variables for the following questions: 
    count_for_questions = 0
    input_list = []
    five_answers_summary = {}
    # Check the type of the paper before asking questions:
    f = open(f'{file_name_system_chart_alloy}.txt', 'a', encoding='utf-8')
    f.write('\n')
    f.write('='*100 + '\n')
    f.write(f'Paper No.{count_for_papers}' + '\n')
    f.write('='*100 + '\n')
    #f.write(paper.split('Abstract')[0] + '\n')
    f.write('Check the type of the paper (Experimental/Practical, Theoretical/Computational or Mixed), this paper belongs to: ')
    input_list.append(prompt_for_paper_classification + '\n' + paper)
    input_list, r = openai_api_function(input_list)
    input_list.append(r)
    type_of_paper = r.split('Reasons: ')[0].strip().split('Category: ')[1].strip()
    f.close()
    
    # Based on the different type of the paper, we will ask different questions:
    if type_of_paper == 'Experimental/Practical':
        Questions_later = prompts_for_experimental_type_papers.copy()
        while True:
            f = open(f'{file_name_system_chart_alloy}.txt', 'a', encoding='utf-8')
            # Dealing with first five questions:
            if count_for_questions == 0:
                f.write(' '*5)
                f.write(type_of_paper + '\n')
                f.write('\n')
                input_list.append(Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                five_answers_summary['property'] = r.split(';')
                five_answers_summary['property'] = [x for x in five_answers_summary['property'] if not check_unimportant_property(x)]
                cleaned_properties_string = ';'.join(five_answers_summary['property'])
                input_list.append(cleaned_properties_string if cleaned_properties_string is not None else r)
                f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions} : Properties are: ' + r + '\n') # Writing the Question and results to text file
            elif count_for_questions == 1:
                f.write('~'*100 + '\n')
                input_list.append(Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                five_answers_summary['structure'] = r.split(';')
                five_answers_summary['structure'] = [x for x in five_answers_summary['structure'] if not check_unimportant_microstructure(x)]
                cleaned_microstructure_string = ';'.join(five_answers_summary['structure'])
                input_list.append(cleaned_microstructure_string)
                f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: structures are: '+ cleaned_microstructure_string + '\n') # Writing the Question and results to text file
            elif count_for_questions == 2:
                f.write('~'*100 + '\n')
                input_list.append(Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                composition = r.split('Key elements addition:')[0].split('Composition:')[1].strip()
                key_elements = r.split('Key elements addition:')[1].strip()
                five_answers_summary['composition&key_elements'] =  composition
                if 'N/A' in key_elements:
                    input_list.append(r)
                    f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: ' + r + '\n')
                else:
                    input_list.append(r)
                    f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: ' + r + '\n')
                    five_answers_summary['composition&key_elements'] += '; ' + key_elements
            elif count_for_questions == 3:
                f.write('~'*100 + '\n')
                prerequisites = f"Prerequisites: Enhanced properties of the alloy: {five_answers_summary['property']}; structure of the alloy: {five_answers_summary['structure']}. "
                input_list.append(prerequisites + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                input_list.append(r)
                sp = r # Save the result for the first table 
                f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n') # Writing the Question and results to text file
            elif count_for_questions == 4:
                f.write('~'*100 + '\n')
                input_list.append(Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                five_answers_summary['processing'] = r.split(';')
                # Manulally remove testing procedure in the Processing list:
                cleaned_processing = ';'.join([x for x in five_answers_summary['processing'] if not check_unimportant_processing(x)])
                input_list.append(cleaned_processing)
                f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}' + ' Original processing methods are: ' + r + '\n' + 'Cleaned processing methods are: '+ cleaned_processing + '\n') # Writing the Question and results to text file
            elif count_for_questions == 5:
                f.write('~'*100 + '\n')
                prerequisites = f"Processing methods previously discussed: {cleaned_processing}. "
                input_list.append(prerequisites + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                input_list.append(r)
                five_answers_summary['processing'] = r.split('The explanations are:')[0].split(': ')[1]
                f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n')
            elif count_for_questions == 6:
                f.write('~'*100 + '\n')
                prerequisites = f"Prerequisites: Processing methods of the alloy: {five_answers_summary['processing']}; structure of the alloy: {five_answers_summary['structure']}. "
                input_list.append(prerequisites + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                input_list.append(r)
                f.write(f'Q{count_for_questions}: ' + prerequisites + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n') # Writing the Question and results to text file
            elif count_for_questions == 7:
                f.write('~'*100 + '\n')
                prerequisites = f"Prerequisites: Composition & key elements (if any) of the alloy: {five_answers_summary['composition&key_elements']}; structure of the alloy: {five_answers_summary['structure']}. "
                input_list.append(prerequisites + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                input_list.append(r)
                ps = r # Save the result for the second table
                f.write(f'Q{count_for_questions}: ' + prerequisites + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n') # Writing the Question and results to text file
            elif count_for_questions == 8:
                f.write('~'*100 + '\n')
                prerequisites = f"Prerequisites: First table for relationsip between processing and structure: {ps}; Second table for relationsip between structure and properties: {sp}. "
                input_list.append(prerequisites + '\n' + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                input_list.append(r)
                f.write('\n' + f'Q{count_for_questions}: ' + prerequisites + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n') # Writing the Question and results to text file
            else:
                f.write('~'*100 + '\n')
                prerequisites = f"Prerequisites: The table: {r}; The number of this paper: {count_for_papers}. "
                input_list.append(prerequisites + '\n' + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                table_saver[f"paper{count_for_papers}"] = r # Save the result for the final table
                citation_saver[f"paper{count_for_papers}"] = filename
                f.write(f'Q{count_for_questions}: ' + prerequisites + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n') # Writing the Question and results to text file
            count_for_questions += 1
            f.close()
            # input_list.clear()
            if count_for_questions == 10: # Number of questions + 1
                break
    elif type_of_paper == 'Theoretical/Computational':
        Questions_later = prompts_for_computational_type_papers.copy()
        while True: 
            f = open(f'{file_name_system_chart_alloy}.txt', 'a', encoding='utf-8')
            # Dealing with first five questions:
            if count_for_questions == 0:
                f.write(' '*5)
                f.write(type_of_paper + '\n')
                f.write('\n')
                input_list.append(Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                # target_property = r
                five_answers_summary['property'] = r.split(';')
                five_answers_summary['property'] = [x for x in five_answers_summary['property'] if not check_unimportant_property(x)]
                cleaned_properties_string = ';'.join(five_answers_summary['property'])
                input_list.append(cleaned_properties_string if cleaned_properties_string is not None else r)
                f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions} : Properties are: ' + r + '\n') # Writing the Question and results to text file
            elif count_for_questions == 1:
                f.write('~'*100 + '\n')
                input_list.append(Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                five_answers_summary['structure'] = r.split(';')
                five_answers_summary['structure'] = [x for x in five_answers_summary['structure'] if not check_unimportant_microstructure(x)]
                cleaned_microstructure_string = ';'.join(five_answers_summary['structure'])
                input_list.append(cleaned_microstructure_string)
                f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: structures are: '+ cleaned_microstructure_string + '\n') # Writing the Question and results to text file
            elif count_for_questions == 2:
                f.write('~'*100 + '\n')
                input_list.append(Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                composition = r.split('Key elements addition:')[0].split('Composition:')[1].strip()
                key_elements = r.split('Key elements addition:')[1].strip()
                five_answers_summary['composition&key_elements'] =  composition
                if 'N/A' in key_elements:
                    input_list.append(r)
                    f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: ' + r + '\n')
                else:
                    five_answers_summary['composition&key_elements'] += '; ' + key_elements
                    input_list.append(r)
                    f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: ' + r + '\n')
            elif count_for_questions == 3:
                f.write('~'*100 + '\n')
                prerequisites = f"Prerequisites: Enhanced properties of the alloy: {five_answers_summary['property']}; structure of the alloy: {five_answers_summary['structure']}."
                input_list.append(prerequisites + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                input_list.append(r)
                sp = r # Save the result for the first table
                f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n') # Writing the Question and results to text file
            elif count_for_questions == 4:
                f.write('~'*100 + '\n')
                prerequisites = f"Prerequisites:Composition & Key elements addition of the alloy: {five_answers_summary['composition&key_elements']};structure of the alloy: {five_answers_summary['structure']}."
                input_list.append(prerequisites + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                input_list.append(r)
                ps = r # Save the result for the second table
                f.write(f'Q{count_for_questions}: ' + prerequisites + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n') # Writing the Question and results to text file
            elif count_for_questions == 5:
                f.write('~'*100 + '\n')
                prerequisites = f"Prerequisites: First table for relationsip between processing and structure: {ps}; Second table for relationsip between structure and properties: {sp}. "
                input_list.append(prerequisites + '\n' + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                input_list.append(r)
                f.write('\n' + f'Q{count_for_questions}: ' + prerequisites + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n') # Writing the Question and results to text file
            else:
                f.write('~'*100 + '\n')
                prerequisites = f"Prerequisites: The table: {r}; The number of this paper: {count_for_papers}. "
                input_list.append(prerequisites + '\n' + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                table_saver[f"paper{count_for_papers}"] = r # Save the result for the final table
                citation_saver[f"paper{count_for_papers}"] = filename
                f.write(f'Q{count_for_questions}: ' + prerequisites + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n') # Writing the Question and results to text file
            count_for_questions += 1
            f.close()
            # input_list.clear()
            if count_for_questions == 7: # Number of questions + 1
                break
    elif type_of_paper == 'Mixed':
        Questions_later = prompts_for_mixed_type_papers.copy()
        while True: 
            f = open(f'{file_name_system_chart_alloy}.txt', 'a', encoding='utf-8')
            # Dealing with first five questions:
            if count_for_questions == 0:
                f.write(' '*5)
                f.write(type_of_paper + '\n')
                f.write('\n')
                input_list.append(Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                five_answers_summary['property'] = r.split(';')
                five_answers_summary['property'] = [x for x in five_answers_summary['property'] if not check_unimportant_property(x)]
                cleaned_properties_string = ';'.join(five_answers_summary['property'])
                input_list.append(cleaned_properties_string if cleaned_properties_string is not None else r)
                f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions} : Properties are: ' + r + '\n') # Writing the Question and results to text file
            elif count_for_questions == 1:
                f.write('~'*100 + '\n')
                input_list.append(Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                five_answers_summary['structure'] = r.split(';')
                five_answers_summary['structure'] = [x for x in five_answers_summary['structure'] if not check_unimportant_microstructure(x)]
                cleaned_microstructure_string = ';'.join(five_answers_summary['structure'])
                input_list.append(cleaned_microstructure_string)
                f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: structures are: '+ cleaned_microstructure_string + '\n') # Writing the Question and results to text file
            elif count_for_questions == 2:
                f.write('~'*100 + '\n')
                input_list.append(Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                composition = r.split('Key elements addition:')[0].split('Composition:')[1].strip()
                key_elements = r.split('Key elements addition:')[1].strip()
                five_answers_summary['composition&key_elements'] =  composition
                if 'N/A' in key_elements:
                    input_list.append(r)
                    f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: ' + r + '\n')
                else:
                    input_list.append(r)
                    five_answers_summary['composition&key_elements'] += '; ' + key_elements
                    f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: ' + r + '\n')
            elif count_for_questions == 3:
                f.write('~'*100 + '\n')
                prerequisites = f"Prerequisites: Enhanced properties of the alloy: {five_answers_summary['property']}; structure of the alloy: {five_answers_summary['structure']}."
                input_list.append(prerequisites + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                input_list.append(r)
                sp = r # Save the result for the first table
                f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n') # Writing the Question and results to text file
            elif count_for_questions == 4:
                f.write('~'*100 + '\n')
                input_list.append(Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                five_answers_summary['processing'] = r.split(';')
                # Manulally remove testing procedure in the Processing list:
                cleaned_processing = ';'.join([x for x in five_answers_summary['processing'] if not check_unimportant_processing(x)])
                input_list.append(cleaned_processing)
                f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}' + ' Original processing methods are: ' + r + '\n' + 'Cleaned processing methods are: '+ cleaned_processing + '\n') # Writing the Question and results to text file
            elif count_for_questions == 5:
                f.write('~'*100 + '\n')
                prerequisites = f"Processing methods previously discussed: {cleaned_processing}. "
                input_list.append(prerequisites + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                input_list.append(r)
                five_answers_summary['processing'] = r.split('The explanations are:')[0].split(': ')[1]
                f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n')
            elif count_for_questions == 6:
                f.write('~'*100 + '\n')
                prerequisites = f"Prerequisites: Processing methods of the alloy: {five_answers_summary['processing']}; structure of the alloy: {five_answers_summary['structure']}."
                input_list.append(prerequisites + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                input_list.append(r)
                f.write(f'Q{count_for_questions}: ' + prerequisites + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n') # Writing the Question and results to text file
            elif count_for_questions == 7:
                f.write('~'*100 + '\n')
                prerequisites = f"Prerequisites: Composition & key elements (if any) of the alloy: {five_answers_summary['composition&key_elements']}; structure of the alloy: {five_answers_summary['structure']}. "
                input_list.append(prerequisites + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                input_list.append(r)
                ps = r # Save the result for the second table
                f.write(f'Q{count_for_questions}: ' + prerequisites + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n') # Writing the Question and results to text file
            elif count_for_questions == 8:
                f.write('~'*100 + '\n')
                prerequisites = f"Prerequisites: First table for relationsip between processing and structure: {ps}; Second table for relationsip between structure and properties: {sp}. "
                input_list.append(prerequisites + '\n' + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                input_list.append(r)
                f.write('\n' + f'Q{count_for_questions}: ' + prerequisites + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n') # Writing the Question and results to text file
            else:
                f.write('~'*100 + '\n')
                prerequisites = f"Prerequisites: The table: {r}; The number of this paper: {count_for_papers}. "
                input_list.append(prerequisites + '\n' + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                table_saver[f"paper{count_for_papers}"] = r # Save the result for the final table
                citation_saver[f"paper{count_for_papers}"] = filename
                f.write(f'Q{count_for_questions}: ' + prerequisites + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n') # Writing the Question and results to text file
            count_for_questions += 1
            f.close()
            # input_list.clear()
            if count_for_questions == 10: # Number of questions + 1
                break
    else:
        Questions_later = prompts_for_mixed_type_papers.copy()
        while True: 
            f = open(f'{file_name_system_chart_alloy}.txt', 'a', encoding='utf-8')
            # Dealing with first five questions:
            if count_for_questions == 0:
                f.write(' '*5)
                f.write(type_of_paper + '\n')
                f.write('\n')
                input_list.append(Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                five_answers_summary['property'] = r.split(';')
                five_answers_summary['property'] = [x for x in five_answers_summary['property'] if not check_unimportant_property(x)]
                cleaned_properties_string = ';'.join(five_answers_summary['property'])
                input_list.append(cleaned_properties_string if cleaned_properties_string is not None else r)
                f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions} : Cleaned properties are: ' + r + '\n') # Writing the Question and results to text file
            elif count_for_questions == 1:
                f.write('~'*100 + '\n')
                input_list.append(Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                five_answers_summary['structure'] = r.split(';')
                five_answers_summary['structure'] = [x for x in five_answers_summary['structure'] if not check_unimportant_microstructure(x)]
                cleaned_microstructure_string = ';'.join(five_answers_summary['structure'])
                input_list.append(cleaned_microstructure_string)
                f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: Cleaned structures are: '+ cleaned_microstructure_string + '\n') # Writing the Question and results to text file
            elif count_for_questions == 2:
                f.write('~'*100 + '\n')
                input_list.append(Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                composition = r.split('Key elements addition:')[0].split('Composition:')[1].strip()
                key_elements = r.split('Key elements addition:')[1].strip()
                five_answers_summary['composition&key_elements'] =  composition
                if 'N/A' in key_elements:
                    input_list.append(r)
                    f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: ' + r + '\n')
                else:
                    input_list.append(r)
                    five_answers_summary['composition&key_elements'] += '; ' + key_elements
                    f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: ' + r + '\n')
            elif count_for_questions == 3:
                f.write('~'*100 + '\n')
                prerequisites = f"Prerequisites: Enhanced properties of the alloy: {five_answers_summary['property']}; structure of the alloy: {five_answers_summary['structure']}."
                input_list.append(prerequisites + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                input_list.append(r)
                sp = r # Save the result for the first table
                f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n') # Writing the Question and results to text file
            elif count_for_questions == 4:
                f.write('~'*100 + '\n')
                input_list.append(Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                five_answers_summary['processing'] = r.split(';')
                # Manulally remove testing procedure in the Processing list:
                cleaned_processing = ';'.join([x for x in five_answers_summary['processing'] if not check_unimportant_processing(x)])
                input_list.append(cleaned_processing)
                f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}' + ' Original processing methods are: : ' + r + '\n' + 'Cleaned processing methods are: : '+ cleaned_processing + '\n') # Writing the Question and results to text file
            elif count_for_questions == 5:
                f.write('~'*100 + '\n')
                prerequisites = f"Processing methods previously discussed: {cleaned_processing}. "
                input_list.append(prerequisites + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                input_list.append(r)
                five_answers_summary['processing'] = r.split('The explanations are:')[0].split(': ')[1]
                f.write(f'Q{count_for_questions}: ' + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n')
            elif count_for_questions == 6:
                f.write('~'*100 + '\n')
                prerequisites = f"Prerequisites: Processing methods of the alloy: {five_answers_summary['processing']}; structure of the alloy: {five_answers_summary['structure']}."
                input_list.append(prerequisites + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                input_list.append(r)
                f.write(f'Q{count_for_questions}: ' + prerequisites + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n') # Writing the Question and results to text file
            elif count_for_questions == 7:
                f.write('~'*100 + '\n')
                prerequisites = f"Prerequisites: Composition & key elements (if any) of the alloy: {five_answers_summary['composition&key_elements']}; structure of the alloy: {five_answers_summary['structure']}. "
                input_list.append(prerequisites + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                input_list.append(r)
                ps = r # Save the result for the second table
                f.write(f'Q{count_for_questions}: ' + prerequisites + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n') # Writing the Question and results to text file
            elif count_for_questions == 8:
                f.write('~'*100 + '\n')
                prerequisites = f"Prerequisites: First table for relationsip between processing and structure: {ps}; Second table for relationsip between structure and properties: {sp}. "
                input_list.append(prerequisites + '\n' + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                input_list.append(r)
                f.write('\n' + f'Q{count_for_questions}: ' + prerequisites + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n') # Writing the Question and results to text file
            else:
                f.write('~'*100 + '\n')
                prerequisites = f"Prerequisites: The table: {r}; The number of this paper: {count_for_papers}. "
                input_list.append(prerequisites + '\n' + Questions_later[count_for_questions])
                input_list, r = openai_api_function(input_list)
                table_saver[f"paper{count_for_papers}"] = r # Save the result for the final table
                citation_saver[f"paper{count_for_papers}"] = filename
                f.write(f'Q{count_for_questions}: ' + prerequisites + Questions_later[count_for_questions] + '\n' + f'A{count_for_questions}: '+ r + '\n') # Writing the Question and results to text file
            count_for_questions += 1
            f.close()
            # input_list.clear()
            if count_for_questions == 10: # Number of questions + 1
                break
    count_for_papers += 1
    

**The next blocks define the hypotheses generation:**

In [26]:
prompt_hypotheses_generation_Corrosion_Internal_Knowledge = '''Background:  
This table compiles findings from corrosion- and passivation-focused research studies on high-entropy alloys (HEAs). Each paper is assigned a unique number, and multiple rows may refer to the same source. HEAs often exhibit strong passivation behavior, yet remain susceptible to localized corrosion (e.g., pitting, crevice corrosion, galvanic attack) due to microstructural heterogeneity and multiphase interactions.

Designing multiphase HEAs that combine **robust passivation behavior** with **resistance to localized corrosion** is a critical and unresolved challenge in the field. Achieving this requires leveraging distinct mechanisms across different phases or electrochemical behaviors that, when combined, produce **synergistic corrosion resistance**—not merely additive effects.

---

**Task Objective:**  
Generate well-reasoned, testable hypotheses that propose **synergistic combinations of corrosion/passivation mechanisms** across different studies. Each hypothesis must:

- Aim to improve both **passivation stability** and **localized corrosion resistance**
- Propose **non-additive**, interaction-based synergies (i.e., effects greater than the sum of the parts)
- Stay grounded in **realistic alloy systems** (preferably within Fe, Co, Ni, Cr, Al, Si, Ti systems; minor additions of others acceptable if justifiable)
- Be **practically testable** using standard techniques (e.g., potentiodynamic polarization, EIS, SEM/EDS, XPS, heat treatment, alloying)

---

**Instructions:**

**Step 1 – Select a Primary Mechanism:**  
From the table, select a mechanism or microstructural feature from one study (Paper [X]) that contributes to either:
- Formation of a highly protective, stable passive film  
- Suppression of localized corrosion initiation (e.g., via uniform phase distribution, grain boundary engineering, oxide composition)

**Step 2 – Identify a Synergistic Partner Mechanism:**  
Choose a **second mechanism** from a different study (Paper [Y]) that operates through a **distinct electrochemical or structural pathway**. This second mechanism should complement the first by either:
- Reinforcing passive film formation (e.g., via secondary phase enrichment, oxide multilayering)
- Blocking or mitigating localized corrosion propagation (e.g., cathodic site deactivation, repassivation support)

The two mechanisms must interact across **different length scales, electrochemical behaviors, or microstructural domains**, such that their combined effect improves corrosion resistance beyond what either mechanism could achieve alone.

---

**Output Requirements:**

Each hypothesis should:
- Be clearly labeled (Hypothesis #1, #2, etc.)
- Cite the mechanisms and their sources using [paper number]
- Provide a **comprehensive explanation** (minimum 400 words) of:
  - The scientific principles behind each selected mechanism
  - Why these mechanisms are expected to interact synergistically
  - How the hypothesis can be tested (e.g., alloy design strategy, corrosion test method)
  - Potential microstructural or electrochemical indicators of synergy

---

**Output Format:**

Hypothesis #1:  
- **Mechanism A (from Paper [X]):** [Brief but specific description]  
- **Mechanism B (from Paper [Y]):** [Brief but specific description]  
- **Synergistic Interaction:** [Detailed explanation of how the two mechanisms complement each other and lead to a non-additive improvement in passivation and localized corrosion resistance. Discuss interaction logic, potential outcomes, and practical validation.]

Repeat for each additional hypothesis, ensuring coverage across the full range of studies provided in the table.
'''# Currently used prompt

prompt_additional_hypotheses = '''Please review the entire table once more. Have you overlooked any additional hypotheses that were not included in your prior response? Respond using the same format as your previous answer.'''

In [27]:
#prompt_hypotheses_generation_Corrosion_Internal_Knowledge = '''Background Information: The provided table compiles data from various recent research studies, each designated by unique paper numbers. Multiple entries sharing the same paper number originate from a single research study. Multiphase alloys can exhibit superior corrosion resistance by leveraging passivation mechanisms and microstructural configurations. However, multiphase alloys are often susceptible to localized corrosion phenomena (e.g., pitting, crevice corrosion) due to galvanic interactions between distinct phases. Thus, the key challenge lies in designing a multiphase alloy system that combines beneficial passivation characteristics from one phase or mechanism with corrosion resistance strategies from another, achieving robust and synergistically enhanced resistance against localized corrosion.

#Task Objective: The primary goal of this research is to design multiphase alloys with superior passivation behavior and high resistance to localized corrosion. Your hypotheses must specifically address this objective.

#Step 1: Begin by carefully examining each entry in the provided table. Identify and select mechanisms explicitly contributing to superior passivation characteristics or localized corrosion resistance in multiphase alloy systems.

#Step 2: Re-examine the provided table to select additional mechanisms from different research studies (different paper numbers) that, when combined with the mechanisms identified in Step 1, could potentially produce synergistic enhancements in corrosion resistance. Additionally, you may leverage foundational or established knowledge from your internal knowledge base (clearly label as 'From Knowledge Base') if you identify mechanisms or concepts not explicitly stated in the table but essential for generating innovative hypotheses. Synergistic interactions occur when mechanisms or microstructural features from distinct sources interact to produce a corrosion resistance effect greater than the sum of their individual contributions—these interactions should reflect multiplicative rather than merely additive effects.

#Detailing Synergistic Hypotheses: Each hypothesis should provide a comprehensive and detailed explanation of how specific mechanisms or microstructural features from distinct papers or from your internal knowledge base could interact synergistically to enhance passivation and resistance to localized corrosion. Clearly distinguish and label insights derived from the provided table versus those from your internal knowledge base. These explanations should not merely summarize findings but must propose novel interactions and innovative combinations not explicitly discussed in the original studies.

#Citations: When referencing data or findings from the table, cite the corresponding source using the format '[number]', representing the specific paper number listed in the table. When using insights from your internal knowledge base, explicitly label them as '(From Knowledge Base)'.

#Ensure thoroughness in identifying and articulating every potential synergistic hypothesis embedded within the provided table and augmented by your internal knowledge base. Each hypothesis must be detailed, well-developed, and exceed 400 words, addressing the research objective comprehensively. Your review and hypothesis generation must span the full range of studies represented in the table, avoiding bias towards any specific subset of papers.

#Answer Format:
#Hypothesis #1:
#- Idea selected from Step 1: [Clearly state the selected mechanism related to passivation or localized corrosion resistance]
#- Idea selected from Step 2: [Clearly state the synergistically complementary mechanism selected from a different paper or explicitly label as '(From Knowledge Base)']
#- Synergistic mechanism: [Provide a detailed explanation proposing the innovative and synergistic interaction, clearly highlighting how this combination enhances passivation and resistance to localized corrosion beyond additive effects, explicitly distinguishing between table-derived and knowledge base-derived ideas]

#''' # Currently used prompt

#prompt_additional_hypotheses_Internal_Knowledge = '''Please review the entire table once more. Have you overlooked any additional hypotheses that were not included in your prior response? Respond using the same format as your previous answer.'''

**Hypotheses generation:**

In [28]:
olson_tables_corrosion = '''
| Composition/key elements | Passivation & Corrosion Mechanism (Composition/key elements & structure)                                                                 | Structure                                      | Passivation & Corrosion Mechanism (structure & Property)                                                                 | Property                                      | Paper Number |
|--------------------------|--------------------------------------------------------------------------------------------------------------------------|-----------------------------------------------|--------------------------------------------------------------------------------------------------------------------------|-----------------------------------------------|--------------|
| Arc melting              | Arc melting yields a single bcc phase via rapid solidification and homogenization (From knowledge base)                    | Single bcc phase                              | Uniform distribution of elements reduces localized corrosion sites (From knowledge base)                                  | Enhanced corrosion resistance                 | [1]          |
| Heat treatment           | Heat treatment enhances equi-atomic AlCrTiV composition through diffusion and phase stabilization (From knowledge base)    | Equi-atomic AlCrTiV composition               | Balanced elemental distribution aids in uniform oxide layer formation (From knowledge base)                               | Presence of Al2O3, TiO2, V2O3, and Cr2O3 oxides | [1]          |
| Surface treatment        | Surface treatment modifies charge distribution differences among elements via selective oxidation (From knowledge base)    | Charge distribution differences among elements| Elemental charge variations influence oxide stability and formation (From text)                                           | Dynamic-passivity contributing to corrosion resistance | [1]          |
| Electrochemical testing  | Electrochemical testing reveals Al shows high dissolution through anodic dissolution processes (From text)                | Al shows high dissolution                     | Al dissolution leads to Al2O3 formation, enhancing passivation (From text)                                                | Presence of Al2O3 oxides                      | [1]          |
| Oxidation processes      | Oxidation processes promote Ti contributes to protective oxide formation via TiO2 layer development (From text)            | Ti contributes to protective oxide formation  | Ti's role in forming stable TiO2 enhances passivation (From text)                                                        | Presence of TiO2 oxides                       | [1]          |
| Surface analysis         | Surface analysis identifies anodic/cathodic sites via reactivity mapping (From text)                                       | Surface reactivity maps indicate anodic/cathodic sites | Mapping helps identify reactive sites for targeted passivation (From text)                                                | Enhanced corrosion resistance                 | [1]          |
| Homogenization treatment at 1100 °C followed by water-quenching; NiCoCrFeMn | The homogenization treatment yields elemental segregation affects passivity via the reduction of compositional gradients (From text) | Elemental segregation affects passivity  | Segregation can cause weak spots in passive films, leading to increased corrosion susceptibility (From text)                  | Enhanced corrosion resistance with Fe/Cr synergy                         | [2]          |
| Homogenization treatment at 1100 °C followed by water-quenching; NiCoCrFeMn | The homogenization treatment yields interfaces influence oxide stability via the creation of uniform interfaces (From text)        | Interfaces influence oxide stability     | Interfaces can act as sites for oxide formation, stabilizing passive films and reducing corrosion (From text)                  | Presence of Fe/Cr oxides                                                 | [2]          |
| Homogenization treatment at 1100 °C followed by water-quenching; NiCoCrFeMn | The homogenization treatment yields atom distribution impacts dissolution via the even distribution of elements (From text)         | Atom distribution impacts dissolution    | Uneven atom distribution can lead to differential dissolution rates, affecting overall corrosion resistance (From text)        | Enhanced corrosion resistance with Fe/Cr synergy                         | [2]          |
| Homogenization treatment at 1100 °C followed by water-quenching; NiCoCrFeMn | The homogenization treatment yields Cr/Fe synergy stabilizes passive films via the uniform distribution of Cr and Fe (From text)    | Cr/Fe synergy stabilizes passive films   | Cr and Fe together form stable oxides that enhance passivation and reduce corrosion (From text)                                | Enhanced corrosion resistance with Fe/Cr synergy                         | [2]          |
| N/A                                       | N/A                                                                                                                            | Localized corrosion linked to Mn presence| Mn presence can disrupt passive film formation, leading to localized corrosion (From text)                                    | Mn deteriorates passivity                                                | [2]          |
| Heat treatment at 1200°C for 5 hours; Mo and Ta | The heat treatment yields multiphase microstructures with BCC and FCC phases via phase transformation and stabilization (From text) | Multiphase microstructures with BCC and FCC phases | BCC and FCC phases provide structural stability and influence passive film formation (From text)                           | Enhanced corrosion resistance with Mo and Ta additions | [3]          |
| Heat treatment at 1200°C for 5 hours; Mo and Ta | The heat treatment promotes elemental segregation of Mo and Ta via diffusion processes (From text)                  | Elemental segregation of Mo and Ta             | Mo and Ta segregation enhances passive film stability and reduces localized corrosion (From text)                         | Enhanced corrosion resistance with Mo and Ta additions | [3]          |
| Heat treatment at 1200°C for 5 hours; Cr       | The heat treatment enhances interfaces with Cr-rich phases via phase boundary formation (From text)                 | Interfaces with Cr-rich phases                 | Cr-rich interfaces contribute to the formation of a protective Cr oxide layer (From text)                                 | Enhanced corrosion resistance with Mo and Ta additions | [3]          |
| Heat treatment at 1200°C for 5 hours; Phase distribution | The heat treatment influences localized corrosion via phase distribution changes (From text)                        | Localized corrosion influenced by phase distribution | Phase distribution affects localized corrosion resistance by altering electrochemical activity (From text)                 | Enhanced corrosion resistance with Mo and Ta additions | [3]          |
| Heat treatment at 1200°C for 5 hours           | The heat treatment affects atom distribution via diffusion and phase transformation (From text)                     | Atom distribution affects passivation          | Atom distribution impacts passive film uniformity and stability, influencing overall corrosion resistance (From text)     | Enhanced corrosion resistance with Mo and Ta additions | [3]          |
| Grinding with SiC abrasive papers; Mo up to 9.6 at% | The grinding process yields smooth surfaces via the removal of surface irregularities, facilitating uniform passive film formation (From knowledge base). | Interfaces influence passivation                                           | Interfaces between phases act as sites for passive film formation, enhancing protection (From knowledge base)                  | Enhanced corrosion resistance with Mo up to 5 at%                        | [4]          |
| N/A                               | N/A                                                                                                                            | Multiphase microstructures with FCC, BCC-A2, BCC-B2 phases                | Formation of passive films with Cr, Fe, and Mo species enhances corrosion resistance (From text)                              | Presence of Cr, Fe, and Mo species in passive films                      | [4]          |
| N/A                               | N/A                                                                                                                            | Mo segregation at grain boundaries                                         | Mo segregation stabilizes passive films and reduces localized corrosion (From text)                                           | Enhanced corrosion resistance with Mo up to 5 at%                        | [4]          |
| N/A                               | N/A                                                                                                                            | σ phase formation at high Mo content                                       | σ phase formation leads to elemental segregation, increasing localized corrosion (From text)                                  | Elemental segregation leads to localized corrosion                       | [4]          |
| N/A                               | N/A                                                                                                                            | Elemental segregation leads to localized corrosion                         | Segregation disrupts passive film continuity, increasing susceptibility to corrosion (From text)                              | Interfaces influence passivation                                         | [4]          |
| Remelting cycles for uniform composition; AlxCrFeMnCu composition | The remelting cycles yield a multiphase structure via homogenization of elemental distribution, promoting phase stability (From text)                           | Multiphase structure includes FCC and BCC phases                         | Different phases provide varied electrochemical properties, affecting passive film formation and stability (From text)                                         | AlxCrFeMnCu composition affects resistance                               | [5]          |
| Remelting cycles for uniform composition; AlxCrFeMnCu composition | The remelting cycles yield Al-Cu rich BCC precipitates via controlled cooling and solidification, enhancing phase separation (From knowledge base)             | Al-Cu rich BCC precipitates                                              | Al-Cu precipitates can lead to localized corrosion due to galvanic effects between precipitates and matrix (From knowledge base)                               | Increased Al content decreases Cr2O3 and increases Al2O3, weakening passive film | [5]          |
| Remelting cycles for uniform composition; AlxCrFeMnCu composition | The remelting cycles yield elemental segregation with Fe, Cr, Mn in dendrites, Al, Cu in interdendrites via differential solubility and cooling rates (From text) | Elemental segregation with Fe, Cr, Mn in dendrites, Al, Cu in interdendrites | Segregation leads to micro-galvanic cells, influencing localized corrosion and passive film integrity (From text)                                              | Presence of Cr2O3 enhances corrosion resistance                          | [5]          |
| Remelting cycles for uniform composition; AlxCrFeMnCu composition | The remelting cycles yield interfaces prone to localized corrosion via phase boundary formation and elemental partitioning (From knowledge base)               | Interfaces prone to localized corrosion                                  | Interfaces act as sites for initiation of localized corrosion due to differences in electrochemical potential (From knowledge base)                            | AlxCrFeMnCu composition affects resistance                               | [5]          |
| Vacuum casting into a water-cooled copper mold; TiHfZrNbx (x = 0.2, 0.3, 0.4) | The vacuum casting process yields a dense passive film via rapid solidification, which enhances the formation of protective oxides (From knowledge base). | Dense passive film                                                        | The dense passive film acts as a barrier, reducing ion penetration and corrosion (From text).                                  | Enhanced corrosion resistance                                           | [6]          |
| Vacuum casting into a water-cooled copper mold; TiHfZrNbx (x = 0.2, 0.3, 0.4) | The vacuum casting process yields Hf, Zr, and Ti oxides via controlled cooling, promoting stable oxide layer formation (From knowledge base).              | Hf, Zr, and Ti oxides dominate                                            | These oxides form stable, protective layers that prevent further oxidation and corrosion (From text).                          | Presence of HfO2, ZrO2, TiO2                                             | [6]          |
| Vacuum casting into a water-cooled copper mold; TiHfZrNbx (x = 0.2, 0.3, 0.4) | The vacuum casting process yields Nb2O5 in HEA0.4 via selective oxidation during solidification (From knowledge base).                                        | Nb2O5 in HEA0.4                                                           | Nb2O5 contributes to the stability and integrity of the passive layer, enhancing corrosion resistance (From text).             | Presence of Nb2O5                                                        | [6]          |
| Vacuum casting into a water-cooled copper mold; TiHfZrNbx (x = 0.2, 0.3, 0.4) | The vacuum casting process yields grain boundaries in HEA0.4 via rapid cooling, which may lead to elemental segregation (From knowledge base).             | Grain boundaries in HEA0.4 show susceptibility to localized corrosion     | Susceptibility to localized corrosion at grain boundaries due to potential segregation or defects (From knowledge base).       | Localized corrosion susceptibility                                       | [6]          |
| Wire-cut electric discharge machining; Cu content | The wire-cut electric discharge machining yields precise sample preparation via controlled material removal (From knowledge base). | Multiphase microstructures with BCC, B2, FCC phases | Formation of passive film due to multiphase presence, enhancing corrosion resistance (From text)                                | Enhanced corrosion resistance due to Cu content | [7]          |
| Wire-cut electric discharge machining; Cu content | The wire-cut electric discharge machining yields accurate interfaces via precise cutting, influencing elemental segregation (From knowledge base). | Elemental segregation at interfaces            | Segregation at interfaces can lead to localized passivation, improving overall corrosion resistance (From knowledge base)       | Enhanced corrosion resistance due to Cu content | [7]          |
| Wire-cut electric discharge machining; Cu content | The wire-cut electric discharge machining yields defined regions via precise machining, affecting localized corrosion (From knowledge base). | Localized corrosion in inter-dendritic regions | Localized corrosion in inter-dendritic regions can be mitigated by passive film formation (From knowledge base)                 | Enhanced corrosion resistance due to Cu content | [7]          |
| N/A                                   | N/A                                                                                                                            | Cu-rich and Fe-rich segregation                | Cu-rich areas promote passive film formation, reducing corrosion rate (From text)                                              | Presence of passive film                      | [7]          |
| N/A                                   | N/A                                                                                                                            | Distribution of Cu affects passivation         | Cu distribution enhances passive film stability, reducing corrosion current (From text)                                        | Cu1.0 shows lowest corrosion current          | [7]          |
| Remelting for chemical homogeneity; Mo addition | The remelting process yields uniform elemental distribution via enhanced mixing, reducing elemental segregation (From text)                                      | Elemental segregation leads to micro-galvanic corrosion                  | Elemental segregation creates potential differences, leading to localized corrosion at phase boundaries (From text)                                             | Interfaces promote localized corrosion                                   | [8]          |
| Remelting for chemical homogeneity; Mo addition | The remelting process yields consistent phase distribution via improved alloy homogeneity, influencing multiphase microstructures (From text)                    | Multiphase microstructures with FCC, B2, and σ phases                    | Multiphase structures provide diverse pathways for ion diffusion, affecting passivation layer formation (From text)                                             | Enhanced corrosion resistance due to Mo addition                         | [8]          |
| Remelting for chemical homogeneity; Mo addition | The remelting process yields stable passive films via uniform atom distribution, enhancing passive film stability (From knowledge base)                         | Atom distribution affects passive film stability                         | Uniform atom distribution enhances passive film stability, reducing defect density and improving corrosion resistance (From knowledge base)                     | Improved passivation with increased Cr and Mo content                    | [8]          |
| N/A                                 | N/A                                                                                                                                                            | Cr and Mo-rich oxides enhance passivation                                | Cr2O3 and MoO3 oxides form a dense, protective passive film, reducing corrosion rates (From text)                                                               | Presence of Cr2O3 and MoO3 oxides                                        | [8]          |
| N/A                                 | N/A                                                                                                                                                            | Interfaces promote localized corrosion                                   | Interfaces between phases act as sites for localized corrosion due to differing electrochemical potentials (From text)                                          | Superior corrosion resistance in Mo0.3 alloy                             | [8]          |
| Cutting into coupons for experiments; Sn addition significantly affects performance and structure | The cutting into coupons yields localized corrosion at FCC/Ni3Sn2 interfaces via exposure of phase boundaries to corrosive environments (From text) | Localized corrosion at FCC/Ni3Sn2 interfaces   | Corrosion initiates at interfaces due to potential differences (From text)                                                    | Localized corrosion at FCC/Ni3Sn2 interfaces       | [9]          |
| Cutting into coupons for experiments; Sn addition significantly affects performance and structure | The cutting into coupons yields elemental segregation at phase interfaces via preparation of samples for detailed analysis (From text)              | Elemental segregation at phase interfaces      | Potential difference at interfaces leads to localized corrosion (From text)                                                   | Localized corrosion at FCC/Ni3Sn2 interfaces       | [9]          |
| N/A                             | N/A                                                                                                                            | Multiphase structure with FCC and Ni3Sn2 phases | Ni3Sn2 phase acts as a cathode, reducing pitting sensitivity of FCC phase (From text)                                         | Enhanced corrosion resistance with increased Sn content | [9]          |
| N/A                             | N/A                                                                                                                            | Sn addition leads to Ni3Sn2 phase formation     | Sn promotes Ni3Sn2 phase, increasing cathode area and reducing pitting sensitivity (From text)                                | Enhanced corrosion resistance with increased Sn content | [9]          |
| N/A                             | N/A                                                                                                                            | Sn distribution affects corrosion resistance    | Increased Sn content enhances passive film formation and corrosion resistance (From text)                                     | Presence of SnO2 in passive film                   | [9]          |
| Remelting for homogeneity; High Cr content in M7C3 phase | The remelting process yields a multiphase structure with FCC and M7C3 phases via uniform distribution of elements and controlled cooling (From text) | Multiphase structure with FCC and M7C3 phases | Formation of stable passivation film due to Cr-rich M7C3 phase enhancing Cr2O3 presence (From text)                              | Presence of Cr2O3 in passivation film         | [10]         |
| Remelting for homogeneity                       | The remelting process yields carbon segregation at grain boundaries via uneven distribution of carbon during solidification (From knowledge base) | Carbon segregation at grain boundaries | Localized corrosion at grain boundaries due to carbon aggregation, affecting passivation film stability (From text)              | Stable passivation film formation             | [10]         |
| Remelting for homogeneity                       | The remelting process yields galvanic corrosion at phase interfaces via formation of distinct phases with different electrochemical potentials (From text) | Galvanic corrosion at phase interfaces | Galvanic coupling between FCC and M7C3 phases, leading to localized corrosion at interfaces (From text)                          | Optimal corrosion resistance at C0.3 composition | [10]         |
| Remelting for homogeneity; High Cr content in M7C3 phase | The remelting process yields Cr-rich carbides via preferential segregation and reaction of Cr with carbon (From text) | Cr-rich carbides                       | Cr-rich carbides contribute to Cr2O3 formation, enhancing passivation film stability (From text)                                 | Presence of Cr2O3 in passivation film         | [10]         |
| Remelting for homogeneity                       | The remelting process yields island-like precipitates at grain boundaries via localized concentration of alloying elements (From knowledge base) | Island-like precipitates at grain boundaries | Precipitates act as barriers to corrosion, enhancing passivation film stability (From knowledge base)                             | Stable passivation film formation             | [10]         |
| High-ratio differential speed rolling (HRDSR) | HRDSR yields grain boundaries that enhance Cr ion transport via increased boundary density (From text)                          | Grain boundaries enhance Cr ion transport | Grain boundaries act as pathways for Cr ions, facilitating the formation of a protective Cr oxide layer (From text)            | Presence of Cr oxide               | [11]         |
| Annealing at various temperatures; Presence of Cr | Annealing yields uniform Cr oxide layer formation via recrystallization and defect reduction (From text)                        | Uniform Cr oxide layer formation   | Cr oxide layer acts as a barrier to prevent further corrosion (From text)                                                       | Enhanced corrosion resistance      | [11]         |
| Annealing at various temperatures   | Annealing yields high-density defects near grain boundaries via incomplete defect removal at lower temperatures (From text)     | High-density defects near grain boundaries | Defects can act as initiation sites for localized corrosion, but also enhance Cr ion transport (From text)                     | Affects corrosion resistance       | [11]         |
| Annealing at various temperatures   | Annealing yields grain size variation via temperature-dependent recrystallization (From text)                                   | Grain size affects corrosion resistance | Smaller grains increase grain boundary density, enhancing Cr ion transport and oxide layer formation (From text)               | Enhanced corrosion resistance      | [11]         |
| Vacuum arc melting in a protective gas environment; Mo additions | The vacuum arc melting yields Ta-rich and Mo-rich growths via controlled solidification and elemental segregation (From text) | Mo additions induce Ta-rich and Mo-rich growths | Mo-rich growths enhance passive film stability by forming MoO4²⁻, which improves corrosion resistance (From text) | Mo addition enhances corrosion resistance via MoO4²⁻ formation          | [12]         |
| Vacuum arc melting in a protective gas environment; Mo additions | The vacuum arc melting yields σ phase via phase transformation during cooling (From text)                                      | σ phase emerges in Mo0.3 and Mo0.5     | σ phase contributes to passive film stability in NaCl, enhancing corrosion resistance (From text)                              | Superior passive film stability in NaCl; Mo0.3 optimal in brine         | [12]         |
| Vacuum arc melting in a protective gas environment; Mo additions | The vacuum arc melting yields elemental segregation via differential cooling rates and solute redistribution (From text)       | Elemental segregation at intercellular regions | Segregation leads to localized corrosion due to galvanic effects between phases (From text)                                    | Mo0.5 degraded in H2SO4                                                 | [12]         |
| Vacuum arc melting in a protective gas environment; Mo additions | The vacuum arc melting yields Cr- and Mo-rich phases via selective phase formation and segregation (From text)                 | Cr- and Mo-rich phases promote localized corrosion | Cr- and Mo-rich phases act as cathodic sites, promoting localized corrosion (From text)                                        | Mo0.5 degraded in H2SO4                                                 | [12]         |
| Isothermal annealing; Ti(Nb,Zr)(Zr,Nb,Ta)(Ta,Ha,V,Cr,Mo,W) | The isothermal annealing yields elemental segregation via diffusion and phase separation (From text)                       | Elemental segregation observed post-annealing       | Segregation can lead to localized galvanic cells, increasing corrosion susceptibility (From text)                      | TiNbTa-based HEAs exhibit enhanced corrosion resistance | [13]         |
| Isothermal annealing; Nb, Ta/V                  | The isothermal annealing yields Nb and Ta/V enrichment via selective diffusion and partitioning (From knowledge base)      | Nb and Ta/V enrichment in TiZrNbTa and TiZrNbV      | Enrichment of Nb and Ta/V can enhance passivation by forming stable oxide layers (From knowledge base)                 | Presence of Nb and Ta/V enrichment                  | [13]         |
| Isothermal annealing; Ti                        | The isothermal annealing yields Ti-rich regions via differential diffusion rates of elements (From knowledge base)         | Ti-rich regions in TiNbTaMo                         | Ti-rich regions may lead to increased reactivity and localized corrosion (From knowledge base)                          | Higher work function correlates with better corrosion resistance | [13]         |
| Isothermal annealing                            | The isothermal annealing yields local chemical fluctuations via atomic rearrangement and diffusion (From text)             | Local chemical fluctuations affect atomic distribution and corrosion behavior | Fluctuations can disrupt uniform passivation layers, leading to increased corrosion (From text)                        | TiNbTa-based HEAs exhibit enhanced corrosion resistance | [13]         |
| Annealing  | Annealing yields uniform Cr distribution via diffusion processes enhancing passivation (From text)                                              | Uniform Cr distribution enhances passivation  | Uniform Cr distribution leads to a more stable and protective passive film (From text) | Presence of Cr2O3 enhances corrosion resistance | [14]         |
| Annealing  | Annealing yields elemental segregation at grain boundaries via differential diffusion rates (From text)                                         | Elemental segregation at grain boundaries     | Segregation can lead to galvanic corrosion at grain boundaries (From text) | N/A                                           | [14]         |
| Aging      | Aging yields L12 and σ phases via precipitation and phase transformation (From text); Mo addition influences σ phase formation (From text)      | Multiphase microstructures include FCC matrix, L12, and σ phases | L12 and σ phases can act as sites for localized corrosion due to galvanic coupling (From text) | N/A                                           | [14]         |
| Aging      | Aging yields Cr and Mo enrichment in σ phase via selective partitioning during phase transformation (From text); Mo addition enhances enrichment (From text) | Cr and Mo enrichment in σ phase               | Cr and Mo enrichment enhances passivation by forming protective oxides (From text) | Presence of Cr2O3 enhances corrosion resistance | [14]         |
| Abrasion with SiC paper up to 1200 grit; Mo, Nb | The abrasion process yields a uniform surface, facilitating accurate electrochemical testing and revealing FCC and Laves phases (From text) | FCC and Laves phases                            | Formation of dual-phase microstructure enhances mechanical stability and corrosion resistance (From text)                        | Enhanced corrosion resistance due to Mo, Nb, Cr                          | [15]         |
| Abrasion with SiC paper up to 1200 grit; Mo, Nb | The abrasion process exposes the surface, allowing for the observation of Mo, Nb enrichment in Laves phase (From text) | Mo, Nb enrichment in Laves phase                | Mo and Nb enrichment promotes formation of protective oxides like Nb₂O₅ and MoO₃, enhancing passivation (From text)              | Presence of Cr(OH)₃, Nb₂O₅, MoO₃ in passive film                         | [15]         |
| Abrasion with SiC paper up to 1200 grit; Mo, Nb | The abrasion process ensures a clean interface, aiding in the study of coherent FCC/Laves interface (From knowledge base) | Coherent FCC/Laves interface                    | Coherent interfaces reduce stress concentration, minimizing localized corrosion (From knowledge base)                            | Enhanced corrosion resistance due to Mo, Nb, Cr                          | [15]         |
| Abrasion with SiC paper up to 1200 grit; Mo, Nb | The abrasion process helps in identifying microgalvanic corrosion at FCC/Laves interface (From knowledge base) | Microgalvanic corrosion at FCC/Laves interface  | Microgalvanic corrosion occurs due to potential differences, but is mitigated by protective passive films (From text) | Enhanced corrosion resistance due to Mo, Nb, Cr                          | [15]         |
| Abrasion with SiC paper up to 1200 grit; Mo, Nb | The abrasion process allows for the analysis of Cr, Mo, Nb distribution, enhancing passive film formation (From text) | Cr, Mo, Nb distribution enhances passive film formation | Distribution of Cr, Mo, Nb facilitates formation of stable, protective passive films (From text) | Presence of Cr(OH)₃, Nb₂O₅, MoO₃ in passive film; Enhanced corrosion resistance due to Mo, Nb, Cr | [15]         |
| Aging treatment  | The aging treatment yields L21 phase distribution via the optimization of phase stability and distribution (From text)          | L21 phase distribution                        | L21 phase distribution affects corrosion by altering the passive film's stability and uniformity (From text)                   | Ti oxides play a major role in the passive film    | [16]         |
| Aging treatment  | The aging treatment reduces elemental segregation between FCC and L21 phases via homogenization of alloying elements (From text) | Elemental segregation between FCC and L21 phases | Elemental segregation influences corrosion resistance by creating localized galvanic cells that can lead to pitting (From text) | 1 h aging time enhances corrosion resistance       | [16]         |
| Aging treatment  | The aging treatment improves interfaces between phases via enhanced phase boundary integrity (From text)                        | Interfaces between phases                     | Interfaces impact passivation by affecting the continuity and integrity of the passive film (From text)                        | Ti oxides play a major role in the passive film    | [16]         |
| Aging treatment  | The aging treatment reduces segregation, enhancing corrosion resistance via uniform passive film formation (From text)          | 1 h aging reduces segregation                 | 1 h aging reduces segregation, enhancing corrosion resistance by promoting a more uniform passive film (From text)             | 1 h aging time enhances corrosion resistance       | [16]         |
| Electrodeposition under various current densities; Fe, Co, Ni, Cr | The electrodeposition yields a single fcc phase via controlled deposition rates and elemental mixing (From text)               | Single fcc phase                              | Homogeneous distribution of elements reduces localized corrosion sites (From text)                                             | Enhanced corrosion resistance at 25 A/dm²     | [17]         |
| Electrodeposition under various current densities; Fe, Co, Ni, Cr | The electrodeposition yields a uniform cellular structure via optimized current density (From text)                            | Uniform cellular structure                    | Dense cellular structure minimizes pathways for corrosive agents (From text)                                                   | Enhanced corrosion resistance at 25 A/dm²     | [17]         |
| Electrodeposition under various current densities; Fe, Co, Ni, Cr | The electrodeposition yields reduced surface cracks at 25 A/dm² via minimized hydrogen evolution (From text)                   | Reduced surface cracks at 25 A/dm²            | Fewer cracks reduce initiation sites for corrosion (From text)                                                                | Enhanced corrosion resistance at 25 A/dm²     | [17]         |
| Electrodeposition under various current densities; Fe, Co, Ni, Cr | The electrodeposition yields elemental distribution affecting surface morphology via current density control (From text)       | Elemental distribution affects surface morphology | Variations in elemental distribution influence passive film formation and stability (From text)                                | Enhanced corrosion resistance at 25 A/dm²     | [17]         |
| Electrodeposition under various current densities; Fe, Co, Ni, Cr | The electrodeposition yields Ni and Cr content influencing cracking via selective deposition (From knowledge base)             | Ni and Cr content influence cracking          | Ni and Cr enhance passivation by forming stable oxides, reducing crack formation (From knowledge base)                         | Enhanced corrosion resistance at 25 A/dm²     | [17]         |
| Electrodeposition under various current densities; Fe, Co, Ni, Cr | The electrodeposition yields a dense and uniform surface at optimal current density via controlled deposition parameters (From text) | Dense and uniform surface at optimal current density | Dense surface limits diffusion of corrosive species, enhancing passivation (From text)                                         | Enhanced corrosion resistance at 25 A/dm²     | [17]         |
| Grinding with SiC paper; Mo content significantly affects performance | The grinding with SiC paper yields atom distribution affects passive film uniformity via surface preparation for uniform passive film formation (From knowledge base) | Atom distribution affects passive film uniformity | Non-uniform atom distribution can lead to uneven passive film, affecting protection (From knowledge base)                 | Mo promotes Cr2O3 enrichment                  | [18]         |
| N/A                                             | N/A                                                                                                                      | Multiphase microstructures with FCC and σ-phase | σ-phase acts as micro-cathode, leading to galvanic corrosion with FCC phase (From text)                                   | Mo content affects corrosion resistance       | [18]         |
| N/A                                             | N/A                                                                                                                      | Mo and Cr segregation                         | Mo promotes Cr2O3 enrichment, enhancing passive film formation (From text)                                                | Presence of Cr2O3 enhances corrosion resistance | [18]         |
| N/A                                             | N/A                                                                                                                      | Interfaces promote localized corrosion        | Interfaces between phases create sites for localized corrosion (From text)                                                | Mo0.6 shows higher resistance than Mo0.4, Mo0.8, Mo1 | [18]         |
| N/A                                             | N/A                                                                                                                      | σ-phase acts as micro-cathode                 | σ-phase acts as micro-cathode, leading to galvanic corrosion with FCC phase (From text)                                   | Mo content affects corrosion resistance       | [18]         |
| Vacuum arc melting in a protective gas environment; Nb enhances corrosion resistance | The vacuum arc melting yields a single-phase BCC structure via controlled solidification and mixing of elements (From text)     | Single-phase BCC structure             | Homogeneous distribution of elements reduces localized corrosion sites (From text)                                             | Enhanced corrosion resistance                                           | [19]         |
| Vacuum arc melting in a protective gas environment; Nb enhances corrosion resistance | The vacuum arc melting yields uniform elemental distribution via homogeneous mixing of alloying elements (From text)            | Uniform elemental distribution         | Prevents elemental segregation, leading to uniform passivation film formation (From text)                                      | Consistent corrosion resistance                                         | [19]         |
| Vacuum arc melting in a protective gas environment; Nb enhances corrosion resistance | The vacuum arc melting yields reduced {011} plane spacing via precise control of alloy composition and cooling rates (From text) | Reduced {011} plane spacing enhances resistance | Limits pathways for corrosive agents, enhancing overall material stability (From text)                                         | Enhanced resistance to localized corrosion                              | [19]         |
| N/A                                             | N/A                                                                                                                            | Layered passivation film               | Formation of stable high-valent oxides like TiO2 and Al2O3 provides a protective barrier (From text)                           | Presence of high-valent oxides like TiO2 and Al2O3                       | [19]         |
| N/A                                             | N/A                                                                                                                            | Nb-induced atomic diffusion            | Nb promotes uphill diffusion of Ti/Al, forming stable passivation films (From text)                                            | Alloy composition includes Nb, Ti, Al, Zr, Mo                            | [19]         |
| Boron doping; Boron (1 to 4 at%)  | The boron doping yields grain refinement via the introduction of boron atoms that increase grain boundary density (From text) | Boron doping causes grain refinement | Grain refinement increases grain boundary density, enhancing diffusion of protective elements to the surface, promoting passivation (From text) | Boron doping improves corrosion resistance by forming a compact passivation film | [20]         |
| Boron doping; Boron (1 to 4 at%)  | The boron doping yields boride precipitates via the reaction of boron with metallic elements at grain boundaries (From text) | Boride precipitates form at grain boundaries | Boride precipitates can lead to Cr-depleted zones, which are susceptible to localized corrosion (From text) | Elemental segregation leads to localized corrosion | [20]         |
| Boron doping; Boron (1 to 4 at%)  | The boron doping yields Cr-depleted zones via the formation of borides that consume chromium (From text) | Cr-depleted zones near borides     | Cr-depleted zones reduce the availability of Cr for forming protective Cr2O3, increasing corrosion susceptibility (From text) | Presence of Cr2O3 in passive film | [20]         |
| Boron doping; Boron (1 to 4 at%)  | The boron doping yields increased grain boundary density via the segregation of boron at grain boundaries (From text) | Increased grain boundary density   | Increased grain boundary density facilitates rapid diffusion of elements like Cr to form a protective film (From text) | Boron doping improves corrosion resistance by forming a compact passivation film | [20]         |
| Boron doping; Boron (1 to 4 at%)  | The boron doping yields elemental segregation via the preferential interaction of boron with specific elements (From knowledge base) | Elemental segregation leads to localized corrosion | Segregation at grain boundaries can create micro-galvanic cells, leading to localized corrosion (From knowledge base) | Elemental segregation leads to localized corrosion | [20]         |
| Aging treatments at 900 °C  | The aging treatment yields L12 precipitates in FCC matrix via controlled thermal exposure (From text)                      | L12 precipitates in FCC matrix     | L12 precipitates enhance localized corrosion resistance by promoting uniform passive film formation (From text)         | Enhanced corrosion resistance with small-sized L12 | [21]         |
| Aging treatments at 900 °C; Co20Cr15Fe20Ni33Al6Ti6 | The aging treatment yields Cr enrichment in passive films via selective diffusion and segregation (From text)              | Cr enrichment in passive films     | Cr enrichment leads to the formation of stable, protective passive films (From text)                                    | Cr-enriched passive films                     | [21]         |
| Aging treatments at 900 °C; Co20Cr15Fe20Ni33Al6Ti6 | The aging treatment yields NiAlTi-enriched oxides on L12 via oxidation during thermal exposure (From text)                 | NiAlTi-enriched oxides on L12      | NiAlTi-enriched oxides contribute to passive film heterogeneity, affecting stability (From text)                        | Presence of oxide and hydroxide species       | [21]         |
| Aging treatments at 900 °C  | The aging treatment yields uniform films with small L12 via complete dissolution and repassivation (From text)             | Uniform films with small L12       | Uniform films ensure consistent protection and rapid repassivation (From text)                                          | Enhanced corrosion resistance with small-sized L12 | [21]         |
| Aging treatments at 900 °C  | The aging treatment yields heterogeneous films with large L12 via incomplete dissolution and nonuniform film formation (From text) | Heterogeneous films with large L12 | Heterogeneous films lead to reduced stability and increased susceptibility to localized corrosion (From text)           | N/A                                           | [21]         |
| Aging treatments at 900 °C  | The aging treatment yields elemental segregation at interfaces via differential diffusion rates (From knowledge base)      | Elemental segregation at interfaces| Elemental segregation can create galvanic cells, affecting corrosion resistance (From knowledge base)                   | N/A                                           | [21]         |
| High-speed laser cladding; Ti addition       | The high-speed laser cladding yields multiphase microstructures with BCC phases via rapid solidification and cooling rates (From knowledge base) | Multiphase microstructures with BCC phases    | BCC phases can lead to galvanic corrosion due to potential differences between phases (From knowledge base)                     | Enhanced corrosion resistance with Ti addition | [22]         |
| High-speed laser cladding; Nb and Zr additions | The high-speed laser cladding yields elemental segregation at grain boundaries via differential cooling and solute redistribution (From knowledge base) | Elemental segregation at grain boundaries     | Segregation can create localized anodic and cathodic sites, promoting localized corrosion (From knowledge base)                  | Interfaces influence localized corrosion       | [22]         |
| High-speed laser cladding; Ni11Zr9 intermetallic compounds | The high-speed laser cladding yields intermetallic compound formation via localized melting and solidification (From knowledge base) | Intermetallic compound formation              | Intermetallics can act as cathodic sites, accelerating corrosion of the surrounding matrix (From knowledge base)                 | Atom distribution affects corrosion            | [22]         |
| High-speed laser cladding; CoCrFeNi composition | The high-speed laser cladding yields atom distribution affecting corrosion via controlled alloying and mixing (From knowledge base) | Atom distribution affects corrosion           | Uniform distribution can enhance passivation by forming continuous protective oxide layers (From knowledge base)                 | Presence of Cr2O3 and TiO2 oxides              | [22]         |
| High-speed laser cladding; CoCrFeNi composition | The high-speed laser cladding yields interfaces influencing localized corrosion via the creation of heterogeneous microstructures (From knowledge base) | Interfaces influence localized corrosion      | Interfaces can disrupt passive film continuity, leading to localized breakdown and corrosion (From knowledge base)               | CoCrFeNi composition                           | [22]         |
| Aging treatment at 700°C; Co40Cr20Ni30Al5Ti5 (at%) | The aging treatment at 700°C yields a dual-phase FCC + L12 structure via precipitation of L12 particles (From text)              | Dual-phase FCC + L12 structure   | L12 phase destabilizes passive films, increasing susceptibility to localized corrosion (From text)                              | Increased Cr(OH)3 content reduces stability   | [23]         |
| Aging treatment at 700°C; Co40Cr20Ni30Al5Ti5 (at%) | The aging treatment at 700°C yields Ni, Al, Ti in L12 phase via elemental partitioning (From text)                               | Ni, Al, Ti in L12 phase          | Elemental partitioning into L12 phase affects passive film composition and stability (From text)                                | Cr2O3-dominated passive films                 | [23]         |
| Aging treatment at 700°C; Co40Cr20Ni30Al5Ti5 (at%) | The aging treatment at 700°C yields Co, Cr in FCC matrix via elemental partitioning (From text)                                  | Co, Cr in FCC matrix             | Cr enrichment in FCC matrix supports Cr2O3 formation, enhancing passivation (From text)                                         | Superior corrosion resistance compared to 304 SS | [23]         |
| Aging treatment at 700°C; Co40Cr20Ni30Al5Ti5 (at%) | The aging treatment at 700°C yields Cr-depleted L12 particles via selective elemental segregation (From text)                    | Cr-depleted L12 particles        | Cr depletion in L12 particles leads to less stable passive films (From text)                                                    | Increased Cr(OH)3 content reduces stability   | [23]         |
| Aging treatment at 700°C; Co40Cr20Ni30Al5Ti5 (at%) | The aging treatment at 700°C yields nanoscale elemental segregation via differential solubility and diffusion (From text)         | Nanoscale elemental segregation  | Segregation causes micro-galvanic coupling, affecting corrosion resistance (From text)                                          | Interfaces prone to galvanic corrosion        | [23]         |
| Aging treatment at 800°C; Co40Cr20Ni30Al5Ti5 (at%) | The aging treatment at 800°C yields interfaces prone to galvanic corrosion via microstructural heterogeneity (From text)          | Interfaces prone to galvanic corrosion | Galvanic corrosion at interfaces between phases accelerates localized corrosion (From text)                                  | Increased Cr(OH)3 content reduces stability   | [23]         |
| Aging treatment at 700°C; Co40Cr20Ni30Al5Ti5 (at%) | The aging treatment at 700°C yields an amorphous passive film via rapid cooling and non-equilibrium conditions (From knowledge base) | Amorphous passive film           | Amorphous structure of passive film provides uniform coverage, enhancing corrosion resistance (From knowledge base)             | Superior corrosion resistance compared to 304 SS | [23]         |
| Cooling in a water-cooled copper mold; FeCoCrNiMo0.1Nb0.1 | The cooling in a water-cooled copper mold yields FCC and Laves phases via rapid solidification and phase separation (From knowledge base) | FCC and Laves phases                          | Micro-galvanic corrosion between phases; Laves phase acts as cathode, FCC as anode (From text)                                 | Superior corrosion resistance in 0.5 M H2SO4  | [24]         |
| Cooling in a water-cooled copper mold; Mo, Nb | The cooling in a water-cooled copper mold yields Mo and Nb enrichment in Laves phase via segregation during solidification (From knowledge base) | Mo and Nb enrichment in Laves phase           | Formation of stable passive film due to Mo and Nb oxides/hydroxides (From text)                                               | Enhanced passive film stability               | [24]         |
| Cooling in a water-cooled copper mold | The cooling in a water-cooled copper mold yields micro-galvanic corrosion at FCC/Laves interfaces via phase boundary formation (From knowledge base) | Micro-galvanic corrosion at FCC/Laves interfaces | Preferential dissolution of FCC phase due to galvanic effect (From text)                                                      | Superior corrosion resistance in 0.5 M H2SO4  | [24]         |
| Cooling in a water-cooled copper mold | The cooling in a water-cooled copper mold yields elemental segregation via differential cooling rates (From knowledge base) | Elemental segregation                         | Localized corrosion at segregated areas; affects passive film formation (From knowledge base)                                  | Enhanced passive film stability               | [24]         |
| Cooling in a water-cooled copper mold | The cooling in a water-cooled copper mold yields preferential attack on FCC phase via phase composition differences (From knowledge base) | Preferential attack on FCC phase              | FCC phase more susceptible to corrosive species; leads to localized corrosion (From text)                                      | Superior corrosion resistance in 0.5 M H2SO4  | [24]         |
| Cooling in a water-cooled copper mold; Cr, Mo, Nb | The cooling in a water-cooled copper mold yields stable passive film on Laves phase via enhanced element distribution (From knowledge base) | Stable passive film on Laves phase            | Protective passive film formation due to Cr, Mo, and Nb oxides/hydroxides (From text)                                          | Presence of Cr, Mo, and Nb oxides/hydroxides  | [24]         |
| Annealing at 1070°C; Al, Cr, Ti | The annealing at 1070°C yields a dual-phase microstructure via phase transformation and diffusion (From text)             | Dual-phase microstructure        | Independent passivation of each phase; distinct oxides form over each phase (From text) | Stable passive films                          | [25]         |
| Annealing at 1070°C; Al, Cr, Ti | The annealing at 1070°C yields distinct FCC and L21 phases via controlled cooling and phase separation (From text)        | Distinct FCC and L21 phases      | Formation of Cr(III) oxides on FCC phase; Ti(IV) and Al(III) oxides on L21 phase (From text) | Presence of Al(III), Cr(III), and Ti(IV) oxides | [25]         |
| Annealing at 1070°C; Al, Cr, Ti | The annealing at 1070°C yields heterophase interfaces via phase boundary formation (From text)                            | Heterophase interfaces           | Localized corrosion initiation at phase interfaces; potential weak points in passive film (From text) | Localized corrosion at phase interfaces       | [25]         |
| Annealing at 1070°C; Al, Cr, Ti | The annealing at 1070°C yields elemental segregation of Al, Cr, Ti via diffusion and partitioning (From text)             | Elemental segregation of Al, Cr, Ti | Enrichment of passivating elements in passive films; enhances film stability (From text) | Stable passive films                          | [25]         |
| Annealing at 1070°C; Al, Cr, Ti | The annealing at 1070°C yields uniform passive film thickness via consistent phase distribution (From text)               | Uniform passive film thickness   | Consistent protection across phases; limits localized breakdown (From text)             | Stable passive films                          | [25]         |
| Annealing at 1070°C; Al, Cr, Ti | The annealing at 1070°C yields localized corrosion at phase interfaces via compositional differences (From text)          | Localized corrosion at phase interfaces | Corrosion initiation at interfaces due to compositional differences; potential for pit formation (From text) | Localized corrosion at phase interfaces       | [25]         |
'''

In [29]:
citations_corrosion = '''paper1: A computational approach for mapping electrochemical activity of multi-principal element alloys
paper2: Bottom-up characterization of electrochemical passivity from simple binary alloys to high entropy alloys
paper3: Corrosion and passivation behaviour of as-cast and heat-treated AlCoCrFeNiX0.5 (X=Mo, Ta) high entropy alloys in 3.5 wt.% NaCl solution
paper4: Corrosion and passivation of AlCrFe2Ni2Mox high-entropy alloys in sulphuric acid
paper5: Effect of heat treatment on the corrosion behavior of eutectic high entropy alloy
paper6: A study on the effect of nitrogen addition on microstructure and corrosion behavior of CoCrFeNiMn high entropy alloy
paper7: Corrosion behaviour of CoCrFeNi high entropy alloy in molten KCl-MgCl₂ eutectic salts at 600 °C
paper8: Electrochemical and passive film behavior of a multi-principal element alloy in chloride environments
paper9: Electrochemical behavior and passive film characterization of NbMoCrTiAl high-entropy alloy in H2SO4 solution
paper10: Pitting corrosion resistance of AlxCoCrFeNi high entropy alloys in chloride solution
paper11: Influence of Mn on the corrosion behavior of AlCoCrFeNiMnx high-entropy alloys in 3.5 wt.% NaCl solution
paper12: Corrosion behavior of refractory high-entropy alloys in supercritical water
paper13: Corrosion behavior and passive film properties of FeCrNiCoCuAlx high-entropy alloys in NaCl solution
paper14: Corrosion and passivation behavior of dual-phase CoCrFeNiNb0.2 high-entropy alloy
paper15: Oxidation resistance of AlCoCrFeNi2.1 eutectic high-entropy alloy at 900 °C
paper16: Influence of composition on corrosion behavior of low-density AlCrTiVx high entropy alloys
paper17: Effect of Sn addition on microstructure and corrosion behavior of AlCoCrFeNiSnx high entropy alloys
paper18: Electrochemical corrosion behavior of CoCrFeMnNi high entropy alloy under different surface states
paper19: Corrosion behavior of FeCoNiCr-based medium-entropy alloys in acidic environment
paper20: The corrosion resistance of novel low-density high-entropy alloys AlCrTiVNb and AlCrTiVNbMo
paper21: Corrosion behavior of AlCoCrFeNi high entropy alloy in simulated acid rain solution
paper22: The corrosion resistance of AlxCrFeCoNi high-entropy alloys in H2SO4 solution
paper23: Electrochemical behavior and passivation of CoCrFeNi high entropy alloy in artificial seawater
paper24: Passivation and pitting corrosion behavior of FeCoCrNi-based high-entropy alloy in chloride solution
'''

# Code A: Adversarial Hypothesis Refinement
Use **A** when you want better hypotheses via adversarial polishing,  
but still in text-only hypothesis space

In [ ]:
import openai

# Define separate API functions for independent Generator, Reviewer, and Discriminator
# Updated for openai>=1.0.0 interface.
from openai import OpenAI
client = OpenAI(api_key='')

# Temperature justification provided in comments below.
def generator_api(messages, T=1):
    # Temperature = 1: Higher creativity and variability, essential for generating novel hypotheses and refinements.
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": message} for message in messages],
        temperature=T
    )
    return messages, response.choices[0].message.content.strip()

def reviewer_api(messages, T=0.7):
    # Temperature = 0.7: Moderate creativity balanced with analytical rigor, ensuring constructive but realistic critique.
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": message} for message in messages],
        temperature=T
    )
    return messages, response.choices[0].message.content.strip()

def discriminator_api(messages, T=0):
    # Temperature = 0: Deterministic output, essential for consistent and objective assessments of hypothesis improvements.
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": message} for message in messages],
        temperature=T
    )
    return messages, response.choices[0].message.content.strip()

# Generator, Reviewer, and Discriminator prompts
prompt_generator = prompt_hypotheses_generation_Corrosion_Internal_Knowledge

prompt_reviewer = """
You are an expert Reviewer evaluating hypotheses specifically for Scientific Robustness & Mechanistic Consistency. 
Critically evaluate the scientific robustness and mechanistic consistency of the following hypothesis. Specifically address:
- Are the proposed mechanisms supported by fundamental corrosion and passivation science?
- Is the interaction between different mechanisms clearly explained and scientifically coherent?
- Highlight explicit mechanistic inconsistencies or unaddressed scientific principles.
Provide detailed and constructive feedback for improvement.

Hypothesis to review:
"""

prompt_discriminator = """
You are an expert Discriminator tasked with determining whether a revised hypothesis demonstrates significant improvement in Scientific Robustness & Mechanistic Consistency compared to its previous version.

Previous hypothesis:

{previous}

Revised hypothesis:

{revised}

Assess explicitly whether the revised hypothesis shows significant improvement and clearly justify your decision.
Respond explicitly with \"Improved\" or \"No significant improvement\" and provide a brief rationale.
"""

# Adversarial loop setup
max_iterations = 4  # Number of adversarial iterations (recommended: 4 to 6)

# Main loop for hypothesis generation and adversarial refinement
for i in range(100):  # Adjust iterations to your practical evaluation capacity
    print(i)
    file_name = 'Corrosion_Adversarial_Hypotheses.txt'
    final_file_name = 'Final_Corrosion_Hypotheses.txt'

    # Open and start a new hypothesis generation with UTF-8 encoding
    with open(file_name, 'a', encoding='utf-8') as f, open(final_file_name, 'a', encoding='utf-8') as final_f:
        f.write(f'\n{"~"*40} START HYPOTHESIS ITERATION {i+1} {"~"*40}\n')

        # Step 1: Generator generates initial hypothesis with Olson tables
        generator_input = f"Tables are: {olson_tables_corrosion}\nCitation resources are: {citations_corrosion}\n{prompt_generator}"
        _, hypothesis = generator_api([generator_input], T=1)
        f.write(f'Initial Hypothesis:\n{hypothesis}\n')

        previous_hypothesis = hypothesis

        # Step 2: Adversarial Loop
        for adv_iter in range(max_iterations):
            f.write(f'\n{"-"*30} ADVERSARIAL ITERATION {adv_iter+1} {"-"*30}\n')

            # Reviewer evaluates hypothesis
            reviewer_input = prompt_reviewer + previous_hypothesis
            _, reviewer_feedback = reviewer_api([reviewer_input], T=0.7)
            f.write(f'Reviewer Feedback:\n{reviewer_feedback}\n')

            # Generator refines hypothesis
            refinement_prompt = f"Based on the following feedback, refine and improve your hypothesis:\n{reviewer_feedback}\n\nPrevious Hypothesis:\n{previous_hypothesis}\n\nRefined Hypothesis:"
            _, refined_hypothesis = generator_api([refinement_prompt], T=1)
            f.write(f'Refined Hypothesis:\n{refined_hypothesis}\n')

            # Discriminator checks improvement
            discriminator_input = prompt_discriminator.format(
                previous=previous_hypothesis,
                revised=refined_hypothesis
            )
            _, discriminator_decision = discriminator_api([discriminator_input], T=0)
            f.write(f'Discriminator Decision:\n{discriminator_decision}\n')

            # Check if improvement is significant
            if "No significant improvement" in discriminator_decision:
                f.write('Convergence reached. Stopping adversarial refinement.\n')
                break

            # Update hypothesis for next iteration
            previous_hypothesis = refined_hypothesis

        # Write only the final refined hypothesis to a separate file
        final_f.write(f'Hypothesis Iteration {i+1}:\n{previous_hypothesis}\n\n')

        f.write(f'\n{"~"*40} END HYPOTHESIS ITERATION {i+1} {"~"*40}\n')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


# Code B: Bulk Raw Hypothesis Generation
Use **B** when you just need a large pool of diverse raw hypotheses quickly  
(you’ll curate later).

In [ ]:
import openai

# Define a simplified API function for generating hypotheses only
from openai import OpenAI
client = OpenAI(api_key='')

def generator_api(messages, T=1):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": message} for message in messages],
        temperature=T
    )
    return messages, response.choices[0].message.content.strip()

# Generator prompt
prompt_generator = prompt_hypotheses_generation_Corrosion_Internal_Knowledge

# Output file for all hypotheses
output_file = 'Final_Corrosion_Hypotheses.txt'

for i in range(100):
    print(i)
    with open(output_file, 'a', encoding='utf-8') as f:
        f.write(f'\n{"~"*40} HYPOTHESIS {i+1} {"~"*40}\n')

        # Generate hypothesis
        generator_input = f"Tables are: {olson_tables_corrosion}\nCitation resources are: {citations_corrosion}\n{prompt_generator}"
        _, hypothesis = generator_api([generator_input], T=1)
        f.write(f'{hypothesis}\n')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


# Code C: Experimental Plan Refinement
Use **C** when you’ve settled on hypotheses and want actionable, reviewer-vetted  
experimental plans grounded in your literature notes, with clear feasibility gating.

In [ ]:
import openai
import os
import time
# Define separate API functions for independent Generator, Reviewer, and Discriminator
# Updated for openai>=1.0.0 interface.
from openai import OpenAI
client = OpenAI(api_key='')


# === AGENT APIs ===
def generator_api(messages, T=1):
    # Temperature = 1: Higher creativity and variability, essential for proposing diverse and novel experimental plans.
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": message} for message in messages],
        temperature=T
    )
    return messages, response.choices[0].message.content.strip()


def reviewer_api(messages, T=0.7):
    # Temperature = 0.7: Moderate creativity balanced with analytical rigor, enabling thoughtful but grounded critique.
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": message} for message in messages],
        temperature=T
    )
    return messages, response.choices[0].message.content.strip()

def discriminator_api(messages, T=0):
    # Temperature = 0: Deterministic and reproducible outputs, necessary for strict yes/no evaluation of improvements.
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": message} for message in messages],
        temperature=T
    )
    return messages, response.choices[0].message.content.strip()

# === PROMPT TEMPLATES ===
prompt_generator = """
You are a materials scientist. Your task is to design a detailed experimental plan to test the following hypothesis about corrosion-resistant multiphase alloys.

Instructions:
- Propose alloy composition(s) with precise wt% or at% values or reasonable compositional ranges.
- Define the processing method (e.g., arc melting, annealing).
- Recommend appropriate testing methods (e.g., potentiodynamic polarization, EIS).
- Recommend characterization techniques (e.g., SEM, TEM, XRD).
- Each decision must be justified in terms of how it tests the mechanisms described in the hypothesis.
- Avoid in-situ TEM and neutron-based experiments.

Output Format:
- Alloy Composition(s):
- Processing Method:
- Testing Methods:
- Characterization Techniques:
- Rationale for Each Step:
"""

prompt_reviewer = """
You are a senior corrosion and materials expert. Evaluate the feasibility of the following experimental plan.

Critique it based on:
- Practicality of alloy synthesis
- Realism of testing and characterization methods
- Coherence between experiment and mechanisms
- Accessibility of tools and experimental cost

Provide:
- Feasibility Score (1-10)
- Strengths
- Concerns
- Suggested Improvements
"""

prompt_discriminator = """
You are an expert adjudicator. Your task is to compare two experimental plans and determine if the revised plan shows significant improvement in feasibility based on previous reviewer feedback.

Previous Plan:
{previous}

Revised Plan:
{revised}

Feedback:
{feedback}

Respond ONLY with:
- "Improved" if the revised plan is significantly better.
- "No significant improvement" if it is not.
Provide a one-line rationale.
"""

# === SETTINGS ===
max_iterations = 4  # Number of adversarial refinement rounds
root_path = "Hypothesis_List"

# === MAIN ADVERSARIAL LOOP ===
for folder_name in os.listdir(root_path):
    folder_path = os.path.join(root_path, folder_name)
    if not os.path.isdir(folder_path):
        continue

    print(f"Processing {folder_name}")
    hypothesis_file = os.path.join(folder_path, "Hypothesis.txt")
    if not os.path.isfile(hypothesis_file):
        print(f" No Hypothesis.txt found in {folder_name}. Skipping.")
        continue

    # Read Hypothesis
    with open(hypothesis_file, 'r', encoding='utf-8') as f:
        hypothesis = f.read().strip()

    # Read Papers
    papers = []
    for fname in os.listdir(folder_path):
        if fname.endswith(".txt") and fname != "Hypothesis.txt":
            with open(os.path.join(folder_path, fname), 'r', encoding='utf-8') as pf:
                papers.append(pf.read().strip())

    papers_text = "\n\n".join(papers)

    # Create log files
    full_log_path = os.path.join(folder_path, "Adversarial_Learning_Log.txt")
    final_output_path = os.path.join(folder_path, "Proposed_Experiment.txt")

    with open(full_log_path, 'w', encoding='utf-8') as log_file, open(final_output_path, 'w', encoding='utf-8') as final_file:
        log_file.write(f"\n{'='*30} STARTING ADVERSARIAL LEARNING {'='*30}\n\n")

        # Step 1: Initial Generation
        initial_prompt = f"Hypothesis:\n{hypothesis}\n\nReference Papers:\n{papers_text}\n\n{prompt_generator}"
        _, experiment_plan = generator_api([initial_prompt], T=1)
        log_file.write(f"Initial Experiment Plan:\n{experiment_plan}\n\n")

        previous_plan = experiment_plan

        # Step 2: Adversarial Loop
        for adv_iter in range(max_iterations):
            log_file.write(f"\n{'-'*20} Adversarial Iteration {adv_iter+1} {'-'*20}\n")

            # Reviewer Evaluation
            reviewer_input = f"Hypothesis:\n{hypothesis}\n\nReference Papers:\n{papers_text}\n\nExperimental Plan:\n{previous_plan}\n\n{prompt_reviewer}"
            _, reviewer_feedback = reviewer_api([reviewer_input], T=0.7)
            log_file.write(f"Reviewer Feedback:\n{reviewer_feedback}\n\n")

            # Generator Refinement
            refinement_prompt = f"Based on the feedback below, revise the experiment plan to improve feasibility.\n\nHypothesis:\n{hypothesis}\n\nReference Papers:\n{papers_text}\n\nFeedback:\n{reviewer_feedback}\n\nPrevious Plan:\n{previous_plan}\n\n{prompt_generator}"
            _, refined_plan = generator_api([refinement_prompt], T=1)
            log_file.write(f"Refined Plan:\n{refined_plan}\n\n")

            # Discriminator Check
            disc_prompt_filled = prompt_discriminator.format(previous=previous_plan, revised=refined_plan, feedback=reviewer_feedback)
            _, discriminator_decision = discriminator_api([disc_prompt_filled], T=0)
            log_file.write(f"Discriminator Decision:\n{discriminator_decision}\n\n")

            if "No significant improvement" in discriminator_decision:
                log_file.write("Convergence reached. No significant improvement.\n")
                break

            previous_plan = refined_plan
            time.sleep(1)  # Small pause for API politeness

        # Final Save
        final_file.write("Final Proposed Experimental Plan:\n\n")
        final_file.write(previous_plan)
        final_file.write("\n")

    print(f"Completed {folder_name}")

print("\n All folders processed.")

Processing 22
Completed 22
Processing 26
Completed 26
Processing 27
Completed 27
Processing 32
Completed 32
Processing 33
Completed 33
Processing 36
Completed 36
Processing 38
Completed 38
Processing 43
Completed 43
Processing 44
Completed 44
Processing 47
Completed 47
Processing 49
Completed 49
Processing 50
Completed 50
Processing 51
Completed 51
Processing 53
Completed 53
Processing 54
Completed 54
Processing 7
Completed 7

 All folders processed.
